<a href="https://colab.research.google.com/github/jongwoonalee/jongwoonalee.github.io/blob/main/bladdder_flexattention_0531_final_%EB%B3%B5%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 1: 라이브러리 및 기본 설정
# ========================================================================


In [ ]:

# 이 셀을 먼저 실행하세요 - 필요한 모든 라이브러리를 import 합니다
import os
import re
import zipfile
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from skimage.filters import threshold_otsu
import time
import random
import math
import pickle
import hashlib
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

print("✅ 모든 라이브러리 import 완료!")

# GPU 설정 및 확인 - RTX 6000 Ada x2 최적화
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    num_gpus = torch.cuda.device_count()
    print(f"🚀 {num_gpus}개의 GPU 발견!")

    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        memory_gb = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"   GPU {i}: {gpu_name} ({memory_gb:.1f}GB)")

    # CUDA 최적화 설정
    torch.backends.cudnn.benchmark = True  # 동일한 입력 크기에 대해 최적화
    torch.cuda.empty_cache()               # GPU 메모리 정리

    print(f"✅ 주 디바이스: {device}")
else:
    device = torch.device("cpu")
    print("⚠️  GPU를 찾을 수 없습니다. CPU 모드로 실행됩니다.")

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

# 재현 가능한 결과를 위한 시드 설정
def set_seed(seed=42):
    """
    모든 랜덤 시드를 고정하여 재현 가능한 결과를 얻습니다.

    Args:
        seed (int): 고정할 시드 값 (기본값: 42)
    """
    random.seed(seed)              # Python 기본 random
    np.random.seed(seed)           # NumPy random
    torch.manual_seed(seed)        # PyTorch CPU random
    torch.cuda.manual_seed(seed)   # PyTorch GPU random (현재 디바이스)
    torch.cuda.manual_seed_all(seed)  # PyTorch 모든 GPU random

    # 완전한 재현성을 위한 설정 (속도가 약간 느려질 수 있음)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 환경 변수로도 시드 설정
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"✅ 모든 랜덤 시드를 {seed}로 고정했습니다.")

# 시드 고정 실행
set_seed(42)

# 메모리 사용량 모니터링 함수
def log_gpu_memory(step_name=""):
    """
    현재 GPU 메모리 사용량을 출력합니다.

    Args:
        step_name (str): 현재 단계 이름 (로그 구분용)
    """
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3  # GB 단위
        reserved = torch.cuda.memory_reserved() / 1024**3    # GB 단위
        max_allocated = torch.cuda.max_memory_allocated() / 1024**3

        print(f"🔍 [{step_name}] GPU 메모리 - "
              f"사용중: {allocated:.2f}GB, "
              f"예약됨: {reserved:.2f}GB, "
              f"최대사용: {max_allocated:.2f}GB")
    else:
        print(f"🔍 [{step_name}] CPU 모드 실행 중")

# 초기 메모리 상태 확인
log_gpu_memory("초기 상태")

print("\n" + "="*80)
print("Part 1 완료: 기본 설정 및 라이브러리 준비 완료!")
print("="*80)

✅ 모든 라이브러리 import 완료!
🚀 1개의 GPU 발견!
   GPU 0: NVIDIA GeForce RTX 4060 (8.0GB)
✅ 주 디바이스: cuda:0
PyTorch 버전: 2.7.0+cu118
CUDA 사용 가능: True
✅ 모든 랜덤 시드를 42로 고정했습니다.
🔍 [초기 상태] GPU 메모리 - 사용중: 0.00GB, 예약됨: 0.00GB, 최대사용: 0.00GB

Part 1 완료: 기본 설정 및 라이브러리 준비 완료!


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 2: 데이터 로딩 및 전처리 함수
# ========================================================================

In [ ]:


# 이 셀을 두 번째로 실행하세요 - 데이터 처리에 필요한 모든 함수들을 정의합니다

def extract_identifier(filename):
    """
    파일명에서 환자 ID를 추출하는 함수

    Args:
        filename (str): 이미지 파일명 (예: "S123-456.jpg")

    Returns:
        str or None: 추출된 환자 ID (예: "S123000456")
        str: 파일 확장자

    Example:
        extract_identifier("S123-456.jpg") → ("S123000456", ".jpg")
    """
    # 파일명과 확장자 분리
    name, ext = os.path.splitext(filename)

    # 대괄호가 있으면 제거 (예: "[comment]" 부분)
    if '[' in name:
        name = name.split('[')[0].strip()

    # 패턴 1: S숫자-숫자 형태 (예: S123-456)
    m1 = re.match(r'^S(\d+)-(\d+)(?:_\d{4}-\d{2}-\d{2})?', name)
    if m1:
        slide = m1.group(1)      # "123"
        patch = m1.group(2)      # "456"

        # 패치 번호를 6자리로 패딩 (앞에 0 추가)
        if len(patch) == 3:
            patch_padded = "000" + patch    # 456 → 000456
        elif len(patch) == 4:
            patch_padded = "00" + patch     # 1456 → 001456
        elif len(patch) == 5:
            patch_padded = "0" + patch      # 12456 → 012456
        else:
            patch_padded = patch            # 이미 6자리면 그대로

        return f"S{slide}{patch_padded}"

    # 패턴 2: S숫자, 형태 (예: S123,)
    m2 = re.match(r'^S(\d+)[,;]', name)
    if m2:
        slide_id = m2.group(1)
        return f"S{slide_id}", ext

    # 패턴 3: S + 6-8자리 숫자 (예: S12345678)
    m3 = re.match(r'^S(\d{8}|\d{7}|\d{6})', name)
    if m3:
        slide_id = m3.group(1)
        return f"S{slide_id}", ext

    # 매칭되지 않으면 None 반환
    return None, ext

def convert_file_id_to_excel_format(file_id):
    """
    파일 ID를 Excel에서 사용하는 형태로 변환

    Args:
        file_id (str): 파일에서 추출한 ID

    Returns:
        str or None: Excel 형태로 변환된 ID

    Example:
        convert_file_id_to_excel_format("S123-456") → "S123000456"
    """
    if file_id is None:
        return None

    file_id = str(file_id).strip()

    # "-"가 포함된 경우 (예: S123-456)
    if "-" in file_id:
        parts = file_id.split("-")
        if len(parts) == 2 and parts[1].isdigit():
            patch = parts[1]

            # 패치 번호를 6자리로 패딩
            if len(patch) == 3:
                padded_number = "000" + patch
            elif len(patch) == 4:
                padded_number = "00" + patch
            elif len(patch) == 5:
                padded_number = "0" + patch
            else:
                padded_number = patch

            return f"{parts[0]}{padded_number}"

    # 이미 S로 시작하는 긴 형태면 그대로 반환
    elif len(file_id) > 3 and file_id.startswith("S"):
        return file_id

    return None

# 데이터 로딩 및 매칭 함수 (여기서는 함수만 정의, 실제 로딩은 다음 셀에서)
def load_and_match_data(zip_path, excel_path, base_dir=None):
    """
    ZIP 파일과 Excel 파일을 매칭하여 환자별 데이터를 구성하는 함수

    Args:
        zip_path (str): 이미지가 들어있는 ZIP 파일 경로
        excel_path (str): 라벨 정보가 들어있는 Excel 파일 경로
        base_dir (str, optional): ZIP 압축 해제할 디렉토리

    Returns:
        dict: 환자별로 구성된 데이터 딕셔너리
        {
            "patient_id": {
                "images": [이미지파일경로들],
                "t_label": T-stage 라벨,
                "recur_label": 재발 라벨,
                "grade": 등급 정보,
                ... 기타 정보
            }
        }
    """
    print("🚀 데이터 로딩 및 매칭 시작...")

    # Excel 파일 읽기
    print("📊 Excel 파일 읽는 중...")
    try:
        df = pd.read_excel(excel_path)
        print(f"   ✅ Excel 파일 로드 완료: {len(df)}개 행")
        print(f"   📋 컬럼들: {list(df.columns)}")
    except Exception as e:
        print(f"   ❌ Excel 파일 읽기 실패: {e}")
        return {}

    # T-stage와 재발 라벨 생성
    print("🏷️  라벨 변환 중...")

        # T-stage 라벨: 1 → 0 (저위험), 2 → 1 (고위험)
    second_column = df.columns[1]  # 두 번째 컬럼 (Subtype)
    df['t_label'] = df[second_column].apply(
        lambda x: 0 if str(x) == '1' else 1
    )
    t_counts = df['t_label'].value_counts()
    print(f"   📈 T-stage 분포: 저위험(0): {t_counts.get(0, 0)}개, 고위험(1): {t_counts.get(1, 0)}개")

    # 재발 라벨: No → 0, Yes → 1
    #if 'Recurrence' in df.columns:
        #df['recur_label'] = df['Recurrence'].apply(
           # lambda x: 0 if str(x).lower() == 'no' else 1
        #)
       # recur_counts = df['recur_label'].value_counts()
      #  print(f"   🔄 재발 분포: 없음(0): {recur_counts.get(0, 0)}개, 있음(1): {recur_counts.get(1, 0)}개")

    # ZIP 파일 처리
    if base_dir is None:
        base_dir = zip_path.replace('.zip', '')

    print(f"📦 ZIP 파일 처리 중: {zip_path}")

    # ZIP 파일이 이미 압축 해제되어 있는지 확인
    if not os.path.exists(base_dir):
        print("   🔄 ZIP 파일 압축 해제 중...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(os.path.dirname(base_dir))
        print("   ✅ ZIP 파일 압축 해제 완료")
    else:
        print("   ✅ 이미 압축 해제된 폴더 발견")

    # 이미지 파일들 찾기
    print("🔍 이미지 파일들 탐색 중...")
    image_files = []
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}

    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                full_path = os.path.join(root, file)
                image_files.append(full_path)

    print(f"   📷 총 {len(image_files)}개의 이미지 파일 발견")

    # 파일명에서 환자 ID 추출 및 매칭
    print("🔗 환자 ID 매칭 중...")
    patient_data = {}
    matched_count = 0
    unmatched_files = []

    for img_path in tqdm(image_files, desc="이미지 파일 처리"):
        filename = os.path.basename(img_path)

        # 파일명에서 환자 ID 추출
        file_id, _ = extract_identifier(filename)
        if file_id is None:
            unmatched_files.append(filename)
            continue

        # Excel 형태로 변환
        excel_id = convert_file_id_to_excel_format(file_id)
        if excel_id is None:
            unmatched_files.append(filename)
            continue

        # Excel에서 해당 환자 찾기
        patient_row = df[df.iloc[:, 0].astype(str).str.contains(excel_id, na=False)]

        if len(patient_row) > 0:
            patient_info = patient_row.iloc[0]
            patient_id = str(patient_info.iloc[0])

            # 환자 데이터 초기화 (처음 발견된 경우)
            if patient_id not in patient_data:
                patient_data[patient_id] = {
                    'images': [],
                    't_label': patient_info.get('t_label', 0),
                    'recur_label': patient_info.get('recur_label', 0),
                    'grade': patient_info.get('Grade', 'Unknown'),
                    't_stage': patient_info.get('T-stage', 'Unknown'),
                    'recurrence': patient_info.get('Recurrence', 'Unknown')
                }

            # 이미지 경로 추가
            patient_data[patient_id]['images'].append(img_path)
            matched_count += 1
        else:
            unmatched_files.append(filename)

    print(f"   ✅ 매칭 완료: {matched_count}개 파일 매칭")
    print(f"   ⚠️  매칭 실패: {len(unmatched_files)}개 파일")
    print(f"   👥 총 환자 수: {len(patient_data)}명")

    # 환자별 이미지 개수 통계
    image_counts = [len(info['images']) for info in patient_data.values()]
    if image_counts:
        print(f"   📊 환자별 이미지 개수 - 평균: {np.mean(image_counts):.1f}개, "
              f"최소: {min(image_counts)}개, 최대: {max(image_counts)}개")

    # 매칭되지 않은 파일 일부 출력 (디버깅용)
    if unmatched_files:
        print(f"   📝 매칭 실패 파일 예시 (처음 5개):")
        for file in unmatched_files[:5]:
            print(f"      - {file}")

    print("✅ 데이터 로딩 및 매칭 완료!")
    return patient_data

print("\n" + "="*80)
print("Part 2 완료: 데이터 처리 함수들 정의 완료!")
print("다음으로 Part 3에서 실제 데이터를 로딩합니다.")
print("="*80)


Part 2 완료: 데이터 처리 함수들 정의 완료!
다음으로 Part 3에서 실제 데이터를 로딩합니다.


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 3: 메가패치 처리 핵심 함수들
# ========================================================================

In [ ]:


# 이 셀을 세 번째로 실행하세요 - FlexAttention의 핵심인 메가패치 처리 함수들을 정의합니다

def split_megapatch_to_patches(megapatch_path, grid_size=4):
    """
    🔪 STEP 1: 1024x1024 메가패치를 4x4=16개의 256x256 패치로 분할

    FlexAttention 논문의 핵심 아이디어:
    - 큰 이미지를 작은 패치들로 나누어 처리
    - 각 패치는 동일한 크기로 정규화

    Args:
        megapatch_path (str): 1024x1024 메가패치 이미지 경로
        grid_size (int): 그리드 크기 (4x4 = 16개 패치, 3x3 = 9개 패치 등)

    Returns:
        list: 16개의 256x256 패치들 (numpy arrays)
        list: 각 패치의 위치 정보 [(i, j), ...]

    Example:
        patches, positions = split_megapatch_to_patches("image.jpg", 4)
        # patches[0]: 좌상단 패치, patches[15]: 우하단 패치
        # positions[0]: (0, 0), positions[15]: (3, 3)
    """
    # 1024x1024 메가패치 읽기
    megapatch = cv2.imread(megapatch_path)
    if megapatch is None:
        raise ValueError(f"❌ 메가패치를 읽을 수 없습니다: {megapatch_path}")

    # BGR → RGB 변환 (OpenCV는 BGR, 우리는 RGB 사용)
    megapatch = cv2.cvtColor(megapatch, cv2.COLOR_BGR2RGB)
    h, w = megapatch.shape[:2]

    # 각 패치 크기 계산: 1024/4 = 256
    patch_size = h // grid_size  # 256x256

    patches = []      # 분할된 패치들을 저장할 리스트
    positions = []    # 각 패치의 위치 정보를 저장할 리스트

    # 4x4 그리드로 분할 (왼쪽 위부터 오른쪽 아래로)
    for i in range(grid_size):        # 세로 방향 (행)
        for j in range(grid_size):    # 가로 방향 (열)
            # 패치의 시작점과 끝점 계산
            y_start = i * patch_size      # 세로 시작 위치
            x_start = j * patch_size      # 가로 시작 위치
            y_end = y_start + patch_size  # 세로 끝 위치
            x_end = x_start + patch_size  # 가로 끝 위치

            # 256x256 패치 추출
            patch = megapatch[y_start:y_end, x_start:x_end]
            patches.append(patch)
            positions.append((i, j))  # (행, 열) 위치 저장

    return patches, positions

def create_three_streams_from_patch(patch_256, megapatch_1024):
    """
    🎯 STEP 2: 각 256x256 패치로부터 3-stream 생성 (수정됨!)

    FlexAttention의 3-stream 구조:
    1. LR (Low Resolution): 빠른 처리를 위한 64x64 저해상도 ← 수정!
    2. HR (High Resolution): 세밀한 분석을 위한 256x256 고해상도
    3. Global: 전체 맥락을 위한 64x64 글로벌 컨텍스트

    Args:
        patch_256 (numpy.ndarray): 256x256 패치 (numpy array)
        megapatch_1024 (numpy.ndarray): 전체 1024x1024 메가패치 (Global 생성용)

    Returns:
        dict: {
            'lr': 64x64 LR 패치,
            'hr': 256x256 HR 패치 (원본),
            'global': 64x64 Global 컨텍스트
        }
    """
    # 1. LR 스트림: 256x256 → 64x64 다운샘플링 (수정된 부분!)
    # INTER_AREA: 축소시 품질이 좋은 보간법
    lr_patch = cv2.resize(patch_256, (64, 64), interpolation=cv2.INTER_AREA)  # ← 이것이 핵심 수정!

    # 2. HR 스트림: 256x256 원본 그대로 사용
    # 세밀한 특징을 분석하기 위해 원본 해상도 유지
    hr_patch = patch_256.copy()

    # 3. Global 스트림: 전체 1024x1024 → 64x64 (매우 작은 overview)
    # 전체적인 구조와 맥락 정보를 제공
    global_context = cv2.resize(megapatch_1024, (64, 64), interpolation=cv2.INTER_AREA)

    return {
        'lr': lr_patch,         # 64x64 LR (빠른 처리용) ← 이제 올바른 크기!
        'hr': hr_patch,         # 256x256 HR (세밀한 분석용)
        'global': global_context # 64x64 Global (맥락 정보용)
    }

def process_megapatch_complete(megapatch_path, patches_per_megapatch=16):
    """
    🚀 STEP 3: 메가패치 전체 처리 파이프라인

    전체 과정:
    1024x1024 메가패치 → 16개 패치로 분할 → 각각 3-stream 생성

    Args:
        megapatch_path (str): 1024x1024 메가패치 경로
        patches_per_megapatch (int): 메가패치당 패치 개수 (16 or 8 등)

    Returns:
        dict: {
            'lr_patches': 16개의 64x64 LR 패치들,
            'hr_patches': 16개의 256x256 HR 패치들,
            'global_tokens': 16개의 64x64 Global 토큰들 (모두 동일),
            'positions': 패치 위치 정보 [(i,j), ...]
        }
    """
    # 원본 메가패치 읽기
    megapatch = cv2.imread(megapatch_path)
    if megapatch is None:
        raise ValueError(f"❌ 메가패치를 읽을 수 없습니다: {megapatch_path}")
    megapatch = cv2.cvtColor(megapatch, cv2.COLOR_BGR2RGB)

    # patches_per_megapatch에 따라 grid_size 결정
    if patches_per_megapatch == 16:
        grid_size = 4    # 4x4 = 16
    elif patches_per_megapatch == 9:
        grid_size = 3    # 3x3 = 9
    elif patches_per_megapatch == 8:
        # 8개는 특별 처리: 4x4에서 8개만 선택
        grid_size = 4
        use_subset = True
    else:
        grid_size = int(math.sqrt(patches_per_megapatch))
        use_subset = False

    # STEP 1: 1024x1024 → 여러개 256x256 패치로 분할
    patches_256, positions = split_megapatch_to_patches(megapatch_path, grid_size)

    # 8개만 사용하는 경우: 체스판 패턴으로 선택 (균등 분포)
    if patches_per_megapatch == 8 and len(patches_256) == 16:
        # 체스판 패턴: (0,0), (0,2), (1,1), (1,3), (2,0), (2,2), (3,1), (3,3)
        selected_indices = []
        for i, (row, col) in enumerate(positions):
            if (row + col) % 2 == 0:  # 체스판 패턴
                selected_indices.append(i)

        # 8개만 선택
        selected_indices = selected_indices[:patches_per_megapatch]
        patches_256 = [patches_256[i] for i in selected_indices]
        positions = [positions[i] for i in selected_indices]

    # STEP 2: 각 패치별로 3-stream 생성
    lr_patches = []       # LR 패치들을 저장할 리스트
    hr_patches = []       # HR 패치들을 저장할 리스트
    global_tokens = []    # Global 토큰들을 저장할 리스트

    for patch_256 in patches_256:
        # 각 패치에 대해 3-stream 생성
        streams = create_three_streams_from_patch(patch_256, megapatch)

        lr_patches.append(streams['lr'])        # 64x64 LR
        hr_patches.append(streams['hr'])        # 256x256 HR
        global_tokens.append(streams['global']) # 64x64 Global

        # 참고: global_tokens는 모두 동일한 전체 이미지의 축소본입니다

    return {
        'lr_patches': lr_patches,     # patches_per_megapatch개 × 64x64
        'hr_patches': hr_patches,     # patches_per_megapatch개 × 256x256
        'global_tokens': global_tokens, # patches_per_megapatch개 × 64x64 (모두 동일)
        'positions': positions        # patches_per_megapatch개 위치 정보
    }

# 테스트 및 시각화 함수
def visualize_patch_splitting(megapatch_path, save_path=None):
    """
    📊 메가패치 분할 과정을 시각화하는 함수 (디버깅 및 확인용)

    Args:
        megapatch_path (str): 시각화할 메가패치 경로
        save_path (str, optional): 결과 이미지 저장 경로
    """
    try:
        # 메가패치 처리
        processed = process_megapatch_complete(megapatch_path)

        # 시각화 설정
        fig, axes = plt.subplots(4, 6, figsize=(18, 12))
        fig.suptitle(f'메가패치 분할 결과: {os.path.basename(megapatch_path)}', fontsize=16)

        # 원본 메가패치 표시
        megapatch = cv2.imread(megapatch_path)
        megapatch = cv2.cvtColor(megapatch, cv2.COLOR_BGR2RGB)
        axes[0, 0].imshow(megapatch)
        axes[0, 0].set_title('원본 메가패치\n(1024x1024)', fontsize=10)
        axes[0, 0].axis('off')

        # 처음 5개 패치의 3-stream 표시
        for i in range(min(5, len(processed['lr_patches']))):
            row = i // 5 + 1
            col_start = (i % 5) + 1

            # LR 패치 (64x64)
            axes[0, col_start].imshow(processed['lr_patches'][i])
            axes[0, col_start].set_title(f'LR {i+1}\n(64x64)', fontsize=8)
            axes[0, col_start].axis('off')

            # HR 패치 (256x256)
            axes[1, col_start].imshow(processed['hr_patches'][i])
            axes[1, col_start].set_title(f'HR {i+1}\n(256x256)', fontsize=8)
            axes[1, col_start].axis('off')

            # Global 토큰 (64x64)
            axes[2, col_start].imshow(processed['global_tokens'][i])
            axes[2, col_start].set_title(f'Global {i+1}\n(64x64)', fontsize=8)
            axes[2, col_start].axis('off')

        # 빈 subplot들 숨기기
        for i in range(4):
            for j in range(6):
                if i > 2 or (i == 0 and j == 0) or (i > 0 and j == 0):
                    continue
                if not axes[i, j].has_data():
                    axes[i, j].axis('off')

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"✅ 시각화 결과 저장: {save_path}")

        plt.show()

        # 통계 정보 출력
        print(f"📊 메가패치 처리 결과:")
        print(f"   - LR 패치 개수: {len(processed['lr_patches'])}개 (각 64x64)")
        print(f"   - HR 패치 개수: {len(processed['hr_patches'])}개 (각 256x256)")
        print(f"   - Global 토큰 개수: {len(processed['global_tokens'])}개 (각 64x64)")
        print(f"   - 위치 정보: {processed['positions'][:5]}... (처음 5개)")

    except Exception as e:
        print(f"❌ 시각화 중 오류 발생: {e}")

print("\n" + "="*80)
print("Part 3 완료: 메가패치 처리 핵심 함수들 정의 완료!")
print("이제 1024x1024 이미지를 16개의 3-stream 패치로 분할할 수 있습니다.")
print("="*80)


Part 3 완료: 메가패치 처리 핵심 함수들 정의 완료!
이제 1024x1024 이미지를 16개의 3-stream 패치로 분할할 수 있습니다.


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 4: Feature Extractor와 HR Selection
# ========================================================================

In [ ]:


# 이 셀을 네 번째로 실행하세요 - ResNet 기반 feature extractor와 논문의 threshold 방식 HR selection을 구현합니다

class ResNetFeatureExtractor(nn.Module):
    """
    🔬 ResNet18 기반 Feature Extractor

    역할:
    - 64x64 이미지용 (LR, Global streams)
    - 256x256 이미지용 (HR stream)
    - 이미지를 고정 크기 feature vector로 변환

    선택지:
    - ResNet18: 안정적이고 검증된 성능 (추천)
    - MobileNetV3: 더 빠르지만 성능 약간 낮음
    """

    def __init__(self, feature_dim=256, model_type='resnet18', pretrained=True):
        """
        Args:
            feature_dim (int): 출력 feature 차원 (256 or 384)
            model_type (str): 사용할 백본 모델 ('resnet18', 'mobilenet', 'efficientnet')
            pretrained (bool): ImageNet 사전훈련 가중치 사용 여부
        """
        super(ResNetFeatureExtractor, self).__init__()

        self.feature_dim = feature_dim
        self.model_type = model_type

        # 백본 모델 선택 및 설정
        if model_type == 'resnet18':
            # ResNet18: 안정적이고 널리 사용됨 (11M parameters)
            resnet = models.resnet18(pretrained=pretrained)
            self.backbone = nn.Sequential(*list(resnet.children())[:-2])  # avgpool, fc 제거
            backbone_out_dim = 512

        elif model_type == 'mobilenet':
            # MobileNetV3-Small: 빠르고 경량 (2.5M parameters)
            from torchvision.models import mobilenet_v3_small
            mobilenet = mobilenet_v3_small(pretrained=pretrained)
            self.backbone = mobilenet.features
            backbone_out_dim = 576

        elif model_type == 'efficientnet':
            # EfficientNet-B0: 효율적이고 성능 좋음 (5.3M parameters)
            from torchvision.models import efficientnet_b0
            efficientnet = efficientnet_b0(pretrained=pretrained)
            self.backbone = efficientnet.features
            backbone_out_dim = 1280

        else:
            raise ValueError(f"지원하지 않는 모델 타입: {model_type}")

        # Global Average Pooling: spatial dimensions를 1x1로 축소
        self.avgpool = nn.AdaptiveAvgPool2d(1)

        # Feature projection: backbone output → 원하는 feature dimension
        self.projection = nn.Sequential(
            nn.Linear(backbone_out_dim, feature_dim),
            nn.LayerNorm(feature_dim),  # Layer Normalization으로 안정성 향상
            nn.ReLU(),
            nn.Dropout(0.1)             # 10% 드롭아웃으로 overfitting 방지
        )

        print(f"✅ {model_type.upper()} Feature Extractor 초기화 완료")
        print(f"   - 백본 출력 차원: {backbone_out_dim}")
        print(f"   - 최종 feature 차원: {feature_dim}")

    def forward(self, x):
        """
        Forward pass: 이미지 배치를 feature vectors로 변환

        Args:
            x: [batch_size, 3, H, W] - RGB 이미지 배치
               H, W는 64 (LR, Global) 또는 256 (HR)

        Returns:
            [batch_size, feature_dim] - 추출된 feature vectors
        """
        # 1. 백본 네트워크를 통한 feature map 추출
        features = self.backbone(x)      # [B, C, H', W'] - 예: [B, 512, H'/32, W'/32]

        # 2. Global Average Pooling으로 spatial dimensions 축소
        pooled = self.avgpool(features)  # [B, C, 1, 1]

        # 3. Flatten: [B, C, 1, 1] → [B, C]
        flattened = pooled.view(pooled.size(0), -1)  # [B, backbone_out_dim]

        # 4. Projection을 통해 원하는 차원으로 변환
        projected = self.projection(flattened)       # [B, feature_dim]

        return projected


class ThresholdBasedHRSelector(nn.Module):
    """
    🎯 논문의 정확한 방식: Threshold 기반 HR Feature Selection

    FlexAttention 논문의 핵심 아이디어:
    - LR attention scores에서 threshold를 계산
    - Threshold 이상인 패치들만 HR로 선택
    - 약 10% 정도가 선택되도록 동적 조정
    - Top-K 고정 선택이 아닌 실제 중요도 기반 선택
    """

    def __init__(self, target_selection_ratio=0.1, min_patches=1, max_patches=4):
        """
        Args:
            target_selection_ratio (float): 목표 선택 비율 (0.1 = 약 10%)
            min_patches (int): 최소 선택 패치 개수 (너무 적으면 강제 선택)
            max_patches (int): 최대 선택 패치 개수 (너무 많으면 제한)
        """
        super(ThresholdBasedHRSelector, self).__init__()
        self.target_selection_ratio = target_selection_ratio
        self.min_patches = min_patches
        self.max_patches = max_patches

        print(f"✅ Threshold 기반 HR Selector 초기화")
        print(f"   - 목표 선택 비율: {target_selection_ratio*100:.1f}%")
        print(f"   - 선택 범위: {min_patches}~{max_patches}개")

    def forward(self, lr_attention_scores, hr_features):
        """
        Threshold 기반으로 중요한 HR features만 선택

        Args:
            lr_attention_scores: [batch_size, 16] - LR patches의 attention scores
            hr_features: [batch_size, 16, feature_dim] - HR patch features

        Returns:
            selected_hr_features: [batch_size, max_patches, feature_dim] - 선택된 HR features
            selection_masks: [batch_size, 16] - binary selection mask (시각화용)
            thresholds: [batch_size] - 사용된 threshold 값들 (분석용)
        """
        batch_size, num_patches, feature_dim = hr_features.shape

        selected_hr_features = []  # 선택된 HR features를 저장할 리스트
        selection_masks = []       # 선택 마스크를 저장할 리스트
        thresholds = []           # 사용된 threshold들을 저장할 리스트

        # 배치의 각 샘플에 대해 개별 처리
        for b in range(batch_size):
            att_scores = lr_attention_scores[b]  # [16] - 이 샘플의 attention scores

            # Step 1: Adaptive threshold 계산
            threshold = self._compute_adaptive_threshold(att_scores)

            # Step 2: Threshold 적용하여 패치 선택
            mask = att_scores > threshold
            selected_indices = torch.where(mask)[0]  # threshold 이상인 패치들의 인덱스

            num_selected = len(selected_indices)

            # Step 3: 선택된 패치 수 검증 및 조정
            if num_selected < self.min_patches:
                # 너무 적게 선택된 경우: 강제로 최소 개수만큼 선택
                _, top_indices = torch.topk(att_scores, self.min_patches)
                selected_indices = top_indices
                threshold = att_scores[top_indices[-1]]  # 새로운 threshold

            elif num_selected > self.max_patches:
                # 너무 많이 선택된 경우: 상위 max_patches개만 선택
                selected_scores = att_scores[selected_indices]
                _, top_within_selected = torch.topk(selected_scores, self.max_patches)
                selected_indices = selected_indices[top_within_selected]
                threshold = att_scores[selected_indices[-1]]  # 새로운 threshold

            # Step 4: 선택된 HR features 추출
            selected_features = hr_features[b, selected_indices]  # [num_selected, feature_dim]

            # Step 5: 고정 크기로 패딩 (배치 처리를 위해)
            if len(selected_indices) < self.max_patches:
                padding_size = self.max_patches - len(selected_indices)
                padding = torch.zeros(padding_size, feature_dim, device=hr_features.device)
                selected_features = torch.cat([selected_features, padding], dim=0)

            selected_hr_features.append(selected_features)

            # Step 6: Binary mask 생성 (시각화 및 분석용)
            binary_mask = torch.zeros_like(att_scores)
            if len(selected_indices) > 0:
                binary_mask[selected_indices] = 1.0
            selection_masks.append(binary_mask)

            thresholds.append(threshold)

        # 리스트들을 텐서로 변환
        selected_hr_features = torch.stack(selected_hr_features)  # [B, max_patches, feature_dim]
        selection_masks = torch.stack(selection_masks)            # [B, 16]
        thresholds = torch.stack(thresholds)                      # [B]

        return selected_hr_features, selection_masks, thresholds

    def _compute_adaptive_threshold(self, attention_scores):
        """
        적응적 threshold 계산 - 여러 방법 중 가장 적절한 것 선택

        Args:
            attention_scores: [16] - 하나의 샘플에 대한 attention scores

        Returns:
            torch.Tensor: 계산된 threshold 값
        """
        try:
            # Method 1: Otsu threshold (이진화에서 사용하는 최적 분할점)
            # 가장 좋은 방법이지만 sklearn 필요
            scores_np = attention_scores.detach().cpu().numpy()
            threshold_val = threshold_otsu(scores_np)
            return torch.tensor(threshold_val, device=attention_scores.device)

        except:
            # Method 2: Percentile-based threshold (Fallback)
            # 상위 target_selection_ratio*2 정도가 선택되도록
            percentile = 1.0 - (self.target_selection_ratio * 2)  # 80th percentile for 10% target
            threshold_val = torch.quantile(attention_scores, percentile)
            return threshold_val

    def get_selection_statistics(self, selection_masks):
        """
        선택 통계 정보 반환 (디버깅 및 모니터링용)

        Args:
            selection_masks: [batch_size, 16] - binary selection masks

        Returns:
            dict: 선택 통계 정보
        """
        num_selected_per_sample = selection_masks.sum(dim=1)  # [batch_size]

        stats = {
            'mean_selected': num_selected_per_sample.float().mean().item(),
            'min_selected': num_selected_per_sample.min().item(),
            'max_selected': num_selected_per_sample.max().item(),
            'selection_ratio': (num_selected_per_sample.float() / selection_masks.shape[1]).mean().item(),
            'std_selected': num_selected_per_sample.float().std().item()
        }

        return stats


# Feature Extractor 성능 비교 함수
def compare_feature_extractors():
    """
    🔬 다양한 Feature Extractor들의 성능과 속도 비교
    실제 선택에 도움을 주는 벤치마크
    """
    print("🔬 Feature Extractor 성능 비교")
    print("="*60)

    # 테스트용 가상 데이터
    dummy_lr = torch.randn(4, 3, 64, 64)    # LR 패치들
    dummy_hr = torch.randn(4, 3, 256, 256)  # HR 패치들

    extractors = {
        'ResNet18': ResNetFeatureExtractor(feature_dim=256, model_type='resnet18'),
        'MobileNetV3': ResNetFeatureExtractor(feature_dim=256, model_type='mobilenet'),
        'EfficientNet-B0': ResNetFeatureExtractor(feature_dim=256, model_type='efficientnet')
    }

    results = {}

    for name, extractor in extractors.items():
        print(f"\n📊 {name} 테스트 중...")

        # 파라미터 수 계산
        total_params = sum(p.numel() for p in extractor.parameters())
        trainable_params = sum(p.numel() for p in extractor.parameters() if p.requires_grad)

        # 속도 측정 (LR 패치)
        start_time = time.time()
        with torch.no_grad():
            for _ in range(10):  # 10번 반복 측정
                _ = extractor(dummy_lr)
        lr_time = (time.time() - start_time) / 10

        # 속도 측정 (HR 패치)
        start_time = time.time()
        with torch.no_grad():
            for _ in range(10):  # 10번 반복 측정
                _ = extractor(dummy_hr)
        hr_time = (time.time() - start_time) / 10

        results[name] = {
            'total_params': total_params,
            'trainable_params': trainable_params,
            'lr_time_ms': lr_time * 1000,
            'hr_time_ms': hr_time * 1000
        }

        print(f"   파라미터 수: {total_params/1e6:.1f}M")
        print(f"   LR 처리 속도: {lr_time*1000:.1f}ms")
        print(f"   HR 처리 속도: {hr_time*1000:.1f}ms")

    # 추천 출력
    print(f"\n🎯 추천:")
    print(f"   - 안정성 우선: ResNet18 (검증된 성능)")
    print(f"   - 속도 우선: MobileNetV3 (가장 빠름)")
    print(f"   - 밸런스: EfficientNet-B0 (성능-속도 절충)")
    print(f"   - 2일 안에 완주: ResNet18 또는 MobileNetV3")

    return results

# 사용법 예시
def example_usage():
    """Feature Extractor와 HR Selector 사용 예시"""
    print("💡 사용 예시:")

    # Feature Extractor 생성
    feature_extractor = ResNetFeatureExtractor(
        feature_dim=256,
        model_type='resnet18',  # 'resnet18', 'mobilenet', 'efficientnet' 중 선택
        pretrained=True
    )

    # HR Selector 생성
    hr_selector = ThresholdBasedHRSelector(
        target_selection_ratio=0.1,  # 10% 선택 목표
        min_patches=1,               # 최소 1개
        max_patches=4                # 최대 4개
    )

    print("✅ 모델 컴포넌트들이 준비되었습니다!")

print("\n" + "="*80)
print("Part 4 완료: Feature Extractor와 HR Selector 정의 완료!")
print("ResNet18 vs MobileNet vs EfficientNet 중 선택 가능합니다.")
print("="*80)


Part 4 완료: Feature Extractor와 HR Selector 정의 완료!
ResNet18 vs MobileNet vs EfficientNet 중 선택 가능합니다.


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 5: 실제 데이터 로딩 (로컬 경로)
# ========================================================================

In [ ]:
# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 5: 완전 수정된 데이터 로딩
# ========================================================================

import re
from tqdm import tqdm  # notebook 대신 일반 tqdm 사용


# 🏠 로컬 경로 설정
zip_path = r"C:\Users\ehdwk\Downloads\ExternalUSB_Bladder_240710.zip"
excel_path = r"C:\Users\ehdwk\Downloads\MIL_TURB_240918_Modified.xlsx"
base_dir = r"C:\Users\ehdwk\Downloads\ExternalUSB_Bladder_240710"

# 📁 작업 디렉토리 설정
work_dir = r"C:\Users\ehdwk\Downloads\FlexAttention_Results"
checkpoint_dir = os.path.join(work_dir, "checkpoints")
log_dir = os.path.join(work_dir, "logs")
cache_dir = os.path.join(work_dir, "cache")
result_dir = os.path.join(work_dir, "results")

for directory in [work_dir, checkpoint_dir, log_dir, cache_dir, result_dir]:
    os.makedirs(directory, exist_ok=True)

print(f"🏠 로컬 환경 설정 완료!")

# 🔍 실제 데이터 폴더 찾기
def find_data_folders(base_dir):
    """중첩된 폴더 구조에서 실제 C_TIL, P_TIL 폴더 찾기"""

    # 1차: 직접 확인
    c_til_dir = os.path.join(base_dir, "C_TIL")
    p_til_dir = os.path.join(base_dir, "P_TIL")

    if os.path.exists(c_til_dir) and os.path.exists(p_til_dir):
        return c_til_dir, p_til_dir

    # 2차: 하위 폴더에서 찾기
    for item in os.listdir(base_dir):
        item_path = os.path.join(base_dir, item)
        if os.path.isdir(item_path):
            nested_c_til = os.path.join(item_path, "C_TIL")
            nested_p_til = os.path.join(item_path, "P_TIL")

            if os.path.exists(nested_c_til) and os.path.exists(nested_p_til):
                print(f"🔍 중첩 폴더에서 발견: {item_path}")
                return nested_c_til, nested_p_til

    return None, None

# 🔧 복잡한 파일명에서 환자 ID 추출
def extract_patient_id_advanced(filename):
    """
    실제 작동했던 로직 + 숨김파일 필터
    """
    # 숨김파일 제거
    if filename.startswith('._'):
        return None

    name, ext = os.path.splitext(filename)

    # 대괄호 앞 부분만 사용 (좌표 정보 제거)
    if '[' in name:
        name = name.split('[')[0].strip()

    # 패턴 1: S15-3380 형식
    m1 = re.match(r'^S(\d+)-(\d+)(?:_\d{4}-\d{2}-\d{2})?', name)
    if m1:
        slide = m1.group(1)  # 예: "15"
        patch = m1.group(2)  # 예: "3380"

        # 패치 번호 길이에 따라 패딩 추가
        if len(patch) == 3:
            patch_padded = "000" + patch
        elif len(patch) == 4:
            patch_padded = "00" + patch
        elif len(patch) == 5:
            patch_padded = "0" + patch
        else:
            patch_padded = patch

        return f"S{slide}{patch_padded}"

    # 패턴 2: S16022792,1A 형식
    m2 = re.match(r'^S(\d+)[,;]', name)
    if m2:
        slide_id = m2.group(1)
        return f"S{slide_id}"

    # 패턴 3: S16021286 형식 (이미 완성된 형식)
    m3 = re.match(r'^S(\d{8}|\d{7}|\d{6})', name)
    if m3:
        slide_id = m3.group(1)
        return f"S{slide_id}"

    return None

# 실제 데이터 폴더 찾기
c_til_dir, p_til_dir = find_data_folders(base_dir)

print(f"\n🔍 폴더 확인:")
print(f"   C_TIL: {c_til_dir}")
print(f"   P_TIL: {p_til_dir}")

if c_til_dir and p_til_dir:
    c_images = [f for f in os.listdir(c_til_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    p_images = [f for f in os.listdir(p_til_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"   C_TIL 이미지: {len(c_images)}개")
    print(f"   P_TIL 이미지: {len(p_images)}개")

    # 샘플 파일명 확인
    print(f"\n📋 샘플 파일명:")
    for i, filename in enumerate(c_images[:3]):
        extracted_id = extract_patient_id_advanced(filename)
        print(f"   {filename} → {extracted_id}")

# Excel 파일 로드 및 라벨 확인
try:
    labels_df = pd.read_excel(excel_path)
    print(f"\n✅ Excel 파일 로드: {len(labels_df)}개 행")
    print(f"📋 컬럼: {list(labels_df.columns)}")

    # 샘플 데이터 확인
    print(f"\n📊 샘플 Excel 데이터:")
    for i in range(min(5, len(labels_df))):
        row = labels_df.iloc[i]
        print(f"   {row['Number']}: Subtype={row.get('Subtype', 'N/A')}, T={row.get('T', 'N/A')}, Recur={row.get('Recur', 'N/A')}")

    # Subtype 분포 확인 (수정된 로직)
    print(f"\n🏷️ 라벨 분포 확인:")
    if 'Subtype' in labels_df.columns:
        subtype_counts = labels_df['Subtype'].value_counts()
        print(f"   Subtype 분포: {dict(subtype_counts)}")

        # 올바른 라벨 변환: Subtype 1→0, Subtype 2→1
        t_labels_corrected = []
        for _, row in labels_df.iterrows():
            subtype = row['Subtype']
            if subtype == 1:
                t_label = 0  # 저위험
            elif subtype == 2:
                t_label = 1  # 고위험
            else:
                t_label = 0  # 기본값
            t_labels_corrected.append(t_label)

        print(f"📈 수정된 T-stage 분포: 저위험(0): {t_labels_corrected.count(0)}개, 고위험(1): {t_labels_corrected.count(1)}개")

except Exception as e:
    print(f"❌ Excel 파일 로딩 실패: {e}")
    labels_df = None

# 향상된 이미지-라벨 매칭
def match_images_with_labels_v2(c_til_dir, p_til_dir, labels_df):
    """향상된 이미지-라벨 매칭"""

    if not c_til_dir or not p_til_dir or labels_df is None:
        print("❌ 필요한 데이터가 없습니다.")
        return []

    # Excel ID를 문자열로 변환하여 집합 생성
    excel_ids = set(str(id_val) for id_val in labels_df['Number'].values)
    print(f"📊 Excel ID 수: {len(excel_ids)}")
    print(f"📋 Excel ID 샘플: {list(excel_ids)[:5]}")

    matched_samples = []
    total_images = 0
    matched_images = 0

    # 두 폴더 모두 처리
    for folder_name, data_dir in [("C_TIL", c_til_dir), ("P_TIL", p_til_dir)]:
        if not os.path.exists(data_dir):
            continue

        image_files = [f for f in os.listdir(data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        total_images += len(image_files)
        print(f"\n📁 {folder_name}: {len(image_files)}개 이미지 처리")

        folder_matched = 0

        for filename in tqdm(image_files, desc=f"{folder_name} 매칭"):
            # 파일명에서 환자 ID 추출
            patient_id = extract_patient_id_advanced(filename)

            if patient_id and patient_id in excel_ids:
                # Excel에서 해당 환자 정보 찾기
                patient_row = labels_df[labels_df['Number'] == patient_id]

                if not patient_row.empty:
                    row = patient_row.iloc[0]

                    # T 컬럼만 사용 (간단!)
                    t_value = row.get('T', 1)
                    t_label = 0 if t_value == 1 else 1  # T=1→0(저위험), T=2→1(고위험)

                    # 재발 라벨
                    recur = row.get('Recur', None)
                    recur_label = int(recur) if pd.notna(recur) else None

                    matched_samples.append({
                        'patient_id': patient_id,
                        'image_path': os.path.join(data_dir, filename),
                        't_label': t_label,
                        'recur_label': recur_label,
                        'subtype': t_value,  # T 값 저장
                        'folder': folder_name
                    })

                    matched_images += 1
                    folder_matched += 1

        print(f"   {folder_name} 매칭: {folder_matched}개")

    print(f"\n✅ 전체 매칭 결과: {total_images}개 이미지 중 {matched_images}개 매칭")
    return matched_samples

# 환자별 그룹화 (개선됨)
def group_by_patient_v2(matched_samples):
    """환자별 데이터 그룹화 (개선된 버전)"""

    patient_data = {}

    for sample in matched_samples:
        patient_id = sample['patient_id']

        if patient_id not in patient_data:
            patient_data[patient_id] = {
                't_label': sample['t_label'],
                'recur_label': sample['recur_label'],
                'images': [],
                't_stage': f"T{sample['subtype']}",  # T1 또는 T2
                'recurrence': 'No' if sample['recur_label'] == 0 else 'Yes' if sample['recur_label'] == 1 else 'Unknown'
            }

        patient_data[patient_id]['images'].append(sample['image_path'])

    # 통계
    if patient_data:
        image_counts = [len(info['images']) for info in patient_data.values()]
        t_distribution = [info['t_label'] for info in patient_data.values()]

        print(f"\n👥 환자별 데이터:")
        print(f"   총 환자: {len(patient_data)}명")
        print(f"   총 이미지: {sum(image_counts)}개")
        print(f"   환자별 평균 이미지: {np.mean(image_counts):.1f}개")
        print(f"   T-stage 분포: 저위험(0)={t_distribution.count(0)}명, 고위험(1)={t_distribution.count(1)}명")

        # 샘플 환자 정보
        sample_id = list(patient_data.keys())[0]
        sample_info = patient_data[sample_id]
        print(f"\n👤 샘플 환자 ({sample_id}):")
        print(f"   이미지 수: {len(sample_info['images'])}개")
        print(f"   T-stage: {sample_info['t_stage']}")
        print(f"   재발: {sample_info['recurrence']}")

    return patient_data

# 실행
# 실행
print(f"\n🚀 향상된 매칭 시작...")
log_gpu_memory("매칭 전")

try:
    all_samples = match_images_with_labels_v2(c_til_dir, p_til_dir, labels_df)
    patient_data = group_by_patient_v2(all_samples)

    if patient_data and len(patient_data) > 0:  # ← 여기 들여쓰기 수정!
        print(f"\n✅ 성공! {len(patient_data)}명의 환자 데이터 준비 완료")

        # 테스트용 10명으로 제한 (내일 확인 후 전체 사용)
        TEST_MODE = False  # ← 이 한 줄만 바꾸면 됩니다! False로 바꾸면 전체

        if TEST_MODE:
            if len(patient_data) > 10:
                limited_ids = list(patient_data.keys())[:10]
                patient_data = {pid: patient_data[pid] for pid in limited_ids}
                print(f"🔬 테스트 모드: {len(patient_data)}명으로 제한")
        else:
            print(f"🚀 전체 모드: {len(patient_data)}명 모든 환자 사용")

        # 저장
        data_save_path = os.path.join(cache_dir, "patient_data.pkl")
        with open(data_save_path, 'wb') as f:
            pickle.dump(patient_data, f)
        print(f"💾 데이터 저장: {data_save_path}")

    else:
        print("❌ 환자 데이터 생성 실패 - 매칭된 데이터가 없습니다")
        patient_data = {}

except Exception as e:
    print(f"❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()
    patient_data = {}

log_gpu_memory("매칭 후")

print("\n" + "="*80)
print("Part 5 완료: 향상된 데이터 로딩!")
print(f"최종 환자 수: {len(patient_data) if patient_data else 0}명")
print("="*80)

🏠 로컬 환경 설정 완료!
🔍 중첩 폴더에서 발견: C:\Users\ehdwk\Downloads\ExternalUSB_Bladder_240710\ExternalUSB_Bladder_240710

🔍 폴더 확인:
   C_TIL: C:\Users\ehdwk\Downloads\ExternalUSB_Bladder_240710\ExternalUSB_Bladder_240710\C_TIL
   P_TIL: C:\Users\ehdwk\Downloads\ExternalUSB_Bladder_240710\ExternalUSB_Bladder_240710\P_TIL
   C_TIL 이미지: 4261개
   P_TIL 이미지: 4446개

📋 샘플 파일명:
   ._S15-11819,B [d=2.01814,x=18599,y=148794,w=2067,h=2066].png → None
   ._S15-16941 [d=2.01814,x=24799,y=169459,w=2067,h=2067].png → None
   ._S15-16941 [d=2.01814,x=26866,y=169459,w=2066,h=2067].png → None

✅ Excel 파일 로드: 100개 행
📋 컬럼: ['Number', 'T', 'Subtype', 'Recur']

📊 샘플 Excel 데이터:
   S15000922: Subtype=sarc, T=2, Recur=1
   S15003203: Subtype=sarc, T=1, Recur=0
   S15003380: Subtype=0, T=2, Recur=1
   S15004965: Subtype=0, T=1, Recur=0
   S15007775: Subtype=0, T=1, Recur=1

🏷️ 라벨 분포 확인:
   Subtype 분포: {0: np.int64(81), 'sarc': np.int64(7), 'micropapillary': np.int64(4), 'SQ': np.int64(3), 'plasmacytoid': np.int64(2), 'sarcom

C_TIL 매칭: 100%|██████████| 4261/4261 [00:00<00:00, 13226.44it/s]


   C_TIL 매칭: 2194개

📁 P_TIL: 4446개 이미지 처리


P_TIL 매칭: 100%|██████████| 4446/4446 [00:00<00:00, 14893.47it/s]

   P_TIL 매칭: 2193개

✅ 전체 매칭 결과: 8707개 이미지 중 4387개 매칭

👥 환자별 데이터:
   총 환자: 100명
   총 이미지: 4387개
   환자별 평균 이미지: 43.9개
   T-stage 분포: 저위험(0)=64명, 고위험(1)=36명

👤 샘플 환자 (S15011819):
   이미지 수: 94개
   T-stage: T2
   재발: Yes

✅ 성공! 100명의 환자 데이터 준비 완료
🚀 전체 모드: 100명 모든 환자 사용
💾 데이터 저장: C:\Users\ehdwk\Downloads\FlexAttention_Results\cache\patient_data.pkl
🔍 [매칭 후] GPU 메모리 - 사용중: 0.00GB, 예약됨: 0.00GB, 최대사용: 0.00GB

Part 5 완료: 향상된 데이터 로딩!
최종 환자 수: 100명


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 6: 체크포인트 시스템 & Hierarchical Self-Attention
# ========================================================================



In [ ]:

import json
from datetime import datetime

class CheckpointManager:
    """
    🔄 체크포인트 관리 시스템

    기능:
    - 매 epoch마다 모델 상태 자동 저장
    - 훈련 중단시 마지막 지점부터 재시작 가능
    - 최고 성능 모델 별도 저장
    - 훈련 로그 및 통계 저장
    """

    def __init__(self, checkpoint_dir, max_keep=5):
        """
        Args:
            checkpoint_dir (str): 체크포인트 저장 디렉토리
            max_keep (int): 최대 보관할 체크포인트 개수 (오래된 것부터 삭제)
        """
        self.checkpoint_dir = checkpoint_dir
        self.max_keep = max_keep
        self.best_score = 0.0
        self.training_log = []

        # 디렉토리 생성
        os.makedirs(checkpoint_dir, exist_ok=True)
        print(f"📁 체크포인트 매니저 초기화: {checkpoint_dir}")

    def save_checkpoint(self, model, optimizer, scheduler, epoch, fold,
                       train_loss, val_metrics=None, is_best=False):
        """
        체크포인트 저장 (매 epoch마다 호출)

        Args:
            model: 훈련 중인 모델
            optimizer: 옵티마이저
            scheduler: 스케줄러
            epoch: 현재 epoch
            fold: 현재 fold 번호
            train_loss: 훈련 loss
            val_metrics: 검증 메트릭들 (dict)
            is_best: 최고 성능 모델인지 여부
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # 체크포인트 정보
        checkpoint = {
            'epoch': epoch,
            'fold': fold,
            'model_state_dict': model.module.state_dict() if hasattr(model, 'module') else model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'train_loss': train_loss,
            'val_metrics': val_metrics or {},
            'timestamp': timestamp,
            'best_score': self.best_score
        }

        # 정규 체크포인트 저장
        checkpoint_path = os.path.join(
            self.checkpoint_dir,
            f"checkpoint_fold{fold}_epoch{epoch:03d}_{timestamp}.pt"
        )
        torch.save(checkpoint, checkpoint_path)

        # 최신 체크포인트로 링크 (재시작 시 사용)
        latest_path = os.path.join(self.checkpoint_dir, f"latest_fold{fold}.pt")
        torch.save(checkpoint, latest_path)

        # 최고 성능 모델 별도 저장
        if is_best:
            best_path = os.path.join(self.checkpoint_dir, f"best_model_fold{fold}.pt")
            torch.save(checkpoint, best_path)
            self.best_score = val_metrics.get('f1', 0.0) if val_metrics else 0.0
            print(f"🏆 새로운 최고 성능 모델 저장! F1: {self.best_score:.4f}")

        # 훈련 로그 업데이트
        log_entry = {
            'epoch': epoch,
            'fold': fold,
            'train_loss': train_loss,
            'val_metrics': val_metrics or {},
            'timestamp': timestamp
        }
        self.training_log.append(log_entry)

        # 로그 파일 저장
        log_path = os.path.join(self.checkpoint_dir, f"training_log_fold{fold}.json")
        with open(log_path, 'w') as f:
            json.dump(self.training_log, f, indent=2)

        print(f"💾 체크포인트 저장: Fold {fold}, Epoch {epoch}, Loss: {train_loss:.4f}")

        # 오래된 체크포인트 정리
        self._cleanup_old_checkpoints(fold)

    def _cleanup_old_checkpoints(self, fold):
        """오래된 체크포인트 파일들 정리"""
        import glob

        # 해당 fold의 체크포인트 파일들 찾기
        pattern = os.path.join(self.checkpoint_dir, f"checkpoint_fold{fold}_*.pt")
        checkpoints = glob.glob(pattern)

        # 생성 시간 순으로 정렬
        checkpoints.sort(key=os.path.getctime)

        # max_keep 개수를 초과하면 오래된 것부터 삭제
        while len(checkpoints) > self.max_keep:
            old_checkpoint = checkpoints.pop(0)
            try:
                os.remove(old_checkpoint)
                print(f"🗑️  오래된 체크포인트 삭제: {os.path.basename(old_checkpoint)}")
            except:
                pass

    def load_latest_checkpoint(self, fold):
        """
        최신 체크포인트 로딩 (재시작 시 사용)

        Args:
            fold: 로딩할 fold 번호

        Returns:
            dict or None: 체크포인트 데이터, 없으면 None
        """
        latest_path = os.path.join(self.checkpoint_dir, f"latest_fold{fold}.pt")

        if os.path.exists(latest_path):
            checkpoint = torch.load(latest_path, map_location=device)
            print(f"📂 체크포인트 로딩: Fold {fold}, Epoch {checkpoint['epoch']}")
            return checkpoint
        else:
            print(f"📂 체크포인트 없음: Fold {fold} (처음부터 시작)")
            return None


class HierarchicalSelfAttention(nn.Module):
    """
    🎯 FlexAttention 논문의 핵심: Hierarchical Self-Attention

    핵심 아이디어:
    - 일반 Self-Attention: O(n²) - 모든 토큰이 모든 토큰과 상호작용
    - Hierarchical: O(n×M) - 선택된 HR 토큰만 상호작용 (M << n)
    - 계산량 대폭 감소하면서 성능 유지!
    """

    def __init__(self, feature_dim=256, num_heads=4, dropout=0.1):
        """
        Args:
            feature_dim (int): feature 차원 (256 추천, 384는 메모리 많이 사용)
            num_heads (int): attention head 개수 (4 추천, 6은 메모리 많이 사용)
            dropout (float): 드롭아웃 비율
        """
        super(HierarchicalSelfAttention, self).__init__()

        self.feature_dim = feature_dim
        self.num_heads = num_heads
        self.head_dim = feature_dim // num_heads

        # feature_dim이 num_heads로 나누어떨어지는지 확인
        assert feature_dim % num_heads == 0, f"feature_dim({feature_dim})이 num_heads({num_heads})로 나누어떨어지지 않습니다!"

        # 🔵 일반 hidden states용 projections (LR + Global + CLS tokens)
        self.q_proj = nn.Linear(feature_dim, feature_dim)  # Query projection
        self.k_proj = nn.Linear(feature_dim, feature_dim)  # Key projection
        self.v_proj = nn.Linear(feature_dim, feature_dim)  # Value projection

        # 🔴 HR features 전용 projections (논문의 W'_K, W'_V)
        # 중요: HR features는 별도의 projection을 사용!
        self.k_proj_hr = nn.Linear(feature_dim, feature_dim)  # W'_K for HR
        self.v_proj_hr = nn.Linear(feature_dim, feature_dim)  # W'_V for HR

        # 출력 projection
        self.out_proj = nn.Linear(feature_dim, feature_dim)
        self.dropout = nn.Dropout(dropout)
        self.scale = math.sqrt(self.head_dim)  # attention scaling factor

        print(f"✅ Hierarchical Self-Attention 초기화")
        print(f"   - Feature dim: {feature_dim}, Heads: {num_heads}, Head dim: {self.head_dim}")

    def forward(self, hidden_states, selected_hr_features):
        """
        Hierarchical Self-Attention 계산 (논문의 핵심 알고리즘)

        Args:
            hidden_states: [batch_size, N, feature_dim]
                          N = LR tokens + Global tokens + CLS token
            selected_hr_features: [batch_size, M, feature_dim]
                                M = 선택된 HR tokens (보통 1~4개)

        Returns:
            output: [batch_size, N, feature_dim] - 업데이트된 hidden states
            attention_map: [batch_size, N-1] - CLS token의 attention (다음 layer용)
        """
        batch_size, N, _ = hidden_states.shape          # N: LR + Global + CLS 개수
        _, M, _ = selected_hr_features.shape            # M: 선택된 HR 개수

        # 🔵 Step 1: 일반 hidden states에 대한 Q, K, V 계산
        Q = self.q_proj(hidden_states)      # [B, N, D] - Query (어디에 집중할지?)
        K_h = self.k_proj(hidden_states)    # [B, N, D] - Key (나는 이런 정보야)
        V_h = self.v_proj(hidden_states)    # [B, N, D] - Value (실제 전달할 정보)

        # 🔴 Step 2: HR features에 대한 별도 K, V 계산 (논문의 핵심!)
        K_hr = self.k_proj_hr(selected_hr_features)  # [B, M, D] - HR용 Key
        V_hr = self.v_proj_hr(selected_hr_features)  # [B, M, D] - HR용 Value

        # 🔗 Step 3: Key와 Value를 연결 [일반 tokens + HR tokens]
        K_all = torch.cat([K_h, K_hr], dim=1)  # [B, N+M, D] - 모든 Keys
        V_all = torch.cat([V_h, V_hr], dim=1)  # [B, N+M, D] - 모든 Values

        # 🧠 Step 4: Multi-head attention을 위한 reshape
        # [B, seq_len, D] → [B, num_heads, seq_len, head_dim]
        Q = Q.view(batch_size, N, self.num_heads, self.head_dim).transpose(1, 2)
        K_all = K_all.view(batch_size, N+M, self.num_heads, self.head_dim).transpose(1, 2)
        V_all = V_all.view(batch_size, N+M, self.num_heads, self.head_dim).transpose(1, 2)

        # ⚡ Step 5: Attention 계산 - 여기서 계산량 O(N×(N+M))
        # 일반 Self-Attention이라면 O((N+M)²)이지만,
        # Query는 N개뿐이므로 O(N×(N+M)) = O(N²+NM)
        scores = torch.matmul(Q, K_all.transpose(-2, -1)) / self.scale  # [B, H, N, N+M]
        attention_weights = F.softmax(scores, dim=-1)                   # attention 확률
        attention_weights = self.dropout(attention_weights)             # 드롭아웃 적용

        # 🎯 Step 6: Attention 적용하여 정보 집약
        attended = torch.matmul(attention_weights, V_all)  # [B, H, N, head_dim]

        # 🔄 Step 7: Multi-head 결과 합치기
        attended = attended.transpose(1, 2).contiguous()  # [B, N, H, head_dim]
        attended = attended.view(batch_size, N, self.feature_dim)  # [B, N, D]

        # 📤 Step 8: 최종 출력 projection
        output = self.out_proj(attended)  # [B, N, D]

        # 📊 Step 9: 다음 layer용 attention map 추출
        # CLS token (마지막 토큰)이 LR tokens에 주는 attention
        cls_attention = attention_weights[:, :, -1, :N-1]  # [B, H, N-1] - CLS → LR
        attention_map = cls_attention.mean(dim=1)          # [B, N-1] - head 평균

        return output, attention_map


# 메모리 사용량 최적화 함수들
def optimize_memory_usage():
    """메모리 사용량 최적화 설정"""
    if torch.cuda.is_available():
        # 메모리 효율적인 attention 사용 (PyTorch 2.0+)
        try:
            torch.backends.cuda.enable_flash_sdp(True)
            print("✅ Flash Attention 활성화 (메모리 효율성 향상)")
        except:
            print("⚠️  Flash Attention 미지원 (PyTorch 버전 확인)")

        # CUDA 메모리 할당 최적화
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
        print("✅ CUDA 메모리 할당 최적화")

        # 메모리 정리
        torch.cuda.empty_cache()
        print("✅ GPU 메모리 정리 완료")

def log_model_info(model):
    """모델 정보 로깅"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"🔍 모델 정보:")
    print(f"   - 총 파라미터: {total_params:,}개 ({total_params/1e6:.1f}M)")
    print(f"   - 훈련 가능: {trainable_params:,}개 ({trainable_params/1e6:.1f}M)")
    print(f"   - 모델 크기: {total_params * 4 / 1024**2:.1f}MB (float32 기준)")

# 체크포인트 매니저 초기화
checkpoint_manager = CheckpointManager(
    checkpoint_dir=checkpoint_dir,
    max_keep=3  # 최대 3개 체크포인트 보관 (디스크 공간 절약)
)

# 메모리 최적화 실행
optimize_memory_usage()

print("\n" + "="*80)
print("Part 6 완료: 체크포인트 시스템 & Hierarchical Self-Attention 준비 완료!")
print("이제 훈련 중단되어도 마지막 지점부터 재시작 가능합니다!")
print("="*80)

📁 체크포인트 매니저 초기화: C:\Users\ehdwk\Downloads\FlexAttention_Results\checkpoints
✅ Flash Attention 활성화 (메모리 효율성 향상)
✅ CUDA 메모리 할당 최적화
✅ GPU 메모리 정리 완료

Part 6 완료: 체크포인트 시스템 & Hierarchical Self-Attention 준비 완료!
이제 훈련 중단되어도 마지막 지점부터 재시작 가능합니다!


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 7: 완전한 MIL 모델과 Dataset
# ========================================================================

# 이 셀을 일곱 번째로 실행하세요 - 완전한 FlexAttention MIL 모델과 Dataset을 구현합니다


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import random
import pickle
import hashlib
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class FlexAttentionPatientMIL(nn.Module):
    """
    🎯 완전한 FlexAttention Multiple Instance Learning 모델

    전체 구조:
    1. 환자별 여러 메가패치 → 각각 8개 패치 → 3-stream features
    2. LR + Global tokens → Standard Self-Attention layers
    3. LR attention → HR selection → FlexAttention layers
    4. CLS token → Patient-level classification (암 단계/재발 예측)

    계산량 최적화:
    - 메가패치당 16개 → 8개 패치로 감소 (50% 절약)
    - Feature dim 384 → 256로 감소 (33% 절약)
    - FA layers 2개 → 1개로 감소 (50% 절약)
    """

    def __init__(self, feature_dim=256, num_classes=2, num_heads=4,
                 num_sa_layers=1, num_fa_layers=1, dropout=0.1,
                 extractor_type='resnet18'):
        """
        Args:
            feature_dim (int): Feature 차원 (256 추천, 메모리 효율적)
            num_classes (int): 분류 클래스 수 (2: binary classification)
            num_heads (int): Attention head 수 (4 추천, 메모리 효율적)
            num_sa_layers (int): Standard Self-Attention layer 수
            num_fa_layers (int): FlexAttention layer 수
            dropout (float): 드롭아웃 비율
            extractor_type (str): Feature extractor 타입 ('resnet18', 'mobilenet')
        """
        super(FlexAttentionPatientMIL, self).__init__()

        self.feature_dim = feature_dim
        self.num_sa_layers = num_sa_layers
        self.num_fa_layers = num_fa_layers

        print(f"🏗️  FlexAttention MIL 모델 초기화 중...")
        print(f"   - Feature dim: {feature_dim}")
        print(f"   - Attention heads: {num_heads}")
        print(f"   - SA layers: {num_sa_layers}, FA layers: {num_fa_layers}")
        print(f"   - Extractor: {extractor_type}")

        # 🔬 Feature extractors (3개의 서로 다른 해상도용) - 모두 256 차원으로 통일!
        if extractor_type == 'resnet18':
            self.lr_extractor = ResNetFeatureExtractor(feature_dim=feature_dim)    # 256으로 통일
            self.global_extractor = ResNetFeatureExtractor(feature_dim=feature_dim) # 256으로 통일
            self.hr_extractor = ResNetFeatureExtractor(feature_dim=feature_dim)    # 256으로 통일
        elif extractor_type == 'mobilenet':
            self.lr_extractor = MobileNetFeatureExtractor(feature_dim=feature_dim)
            self.global_extractor = MobileNetFeatureExtractor(feature_dim=feature_dim)
            self.hr_extractor = MobileNetFeatureExtractor(feature_dim=feature_dim)
        else:
            raise ValueError(f"지원하지 않는 extractor_type: {extractor_type}")

        # 🎯 CLS token (환자 레벨 분류를 위한 특별한 토큰)
        self.cls_token = nn.Parameter(torch.randn(1, 1, feature_dim))

        # 📍 Positional encoding (토큰 위치 정보)
        # 최대 토큰 수: 환자당 20메가패치 × 8패치 = 160 LR + 20 Global + 1 CLS = 181
        max_tokens = 200  # 여유있게 설정
        self.pos_encoding = nn.Parameter(torch.randn(1, max_tokens, feature_dim))

        # 🧠 Standard Self-Attention layers (LR + Global + CLS만 사용)
        self.sa_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=feature_dim,
                nhead=num_heads,
                dim_feedforward=feature_dim * 4,  # FFN hidden dim
                dropout=dropout,
                batch_first=True,
                norm_first=True  # Pre-LN for better training stability
            ) for _ in range(num_sa_layers)
        ])

        # 🎯 FlexAttention components
        self.hr_selectors = nn.ModuleList([
            ThresholdBasedHRSelector(
                target_selection_ratio=0.1,  # 10% 선택
                min_patches=1,
                max_patches=4
            ) for _ in range(num_fa_layers)
        ])

        self.hierarchical_attentions = nn.ModuleList([
            HierarchicalSelfAttention(feature_dim, num_heads, dropout)
            for _ in range(num_fa_layers)
        ])

        # FlexAttention layer용 FFN과 LayerNorm
        self.fa_ffns = nn.ModuleList([
            nn.Sequential(
                nn.Linear(feature_dim, feature_dim * 4),
                nn.GELU(),  # ReLU보다 더 부드러운 활성화 함수
                nn.Dropout(dropout),
                nn.Linear(feature_dim * 4, feature_dim),
                nn.Dropout(dropout)
            ) for _ in range(num_fa_layers)
        ])

        self.fa_layer_norms = nn.ModuleList([
            nn.LayerNorm(feature_dim) for _ in range(num_fa_layers)
        ])

        # 🏥 최종 분류기 (환자 레벨 예측)
        self.classifier = nn.Sequential(
            nn.Linear(feature_dim, feature_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(feature_dim // 2, num_classes)
        )

        # 가중치 초기화
        self._initialize_weights()

        print(f"✅ FlexAttention MIL 모델 초기화 완료!")

    def _initialize_weights(self):
        """가중치 초기화 (더 안정적인 훈련을 위해)"""
        # CLS token 초기화
        nn.init.trunc_normal_(self.cls_token, std=0.02)

        # Positional encoding 초기화
        nn.init.trunc_normal_(self.pos_encoding, std=0.02)

        # Linear layer 초기화
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.trunc_normal_(module.weight, std=0.02)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)

    def forward(self, lr_features, global_features, hr_features):
        """
        FlexAttention MIL Forward Pass

        Args:
            lr_features: [batch_size, total_lr_patches, feature_dim] - 모든 LR features
            global_features: [batch_size, num_megapatches, feature_dim] - Global features
            hr_features: [batch_size, total_hr_patches, feature_dim] - 모든 HR features

        Returns:
            logits: [batch_size, num_classes] - 환자 레벨 예측
            attention_maps: List[Tensor] - attention maps (시각화용)
            selection_stats: Dict - HR selection 통계 (분석용)
        """
        batch_size = lr_features.shape[0]

        # 📊 입력 데이터 크기 확인 및 메모리 효율적 처리
        max_lr_tokens = min(lr_features.shape[1], 128)    # 최대 128개 LR tokens
        max_global_tokens = min(global_features.shape[1], 16)  # 최대 16개 Global tokens
        max_hr_tokens = min(hr_features.shape[1], 128)    # 최대 128개 HR tokens

        # 메모리 절약을 위해 일부 토큰만 사용
        lr_subset = lr_features[:, :max_lr_tokens]        # [B, ≤128, D]
        global_subset = global_features[:, :max_global_tokens]  # [B, ≤16, D]
        hr_subset = hr_features[:, :max_hr_tokens]        # [B, ≤128, D] (나중에 일부만 선택됨)

        # 🔧 Feature dimension 체크 및 통일 (수정된 부분!)
        assert lr_subset.shape[2] == self.feature_dim, f"LR features dim mismatch: {lr_subset.shape[2]} vs {self.feature_dim}"
        assert global_subset.shape[2] == self.feature_dim, f"Global features dim mismatch: {global_subset.shape[2]} vs {self.feature_dim}"
        assert hr_subset.shape[2] == self.feature_dim, f"HR features dim mismatch: {hr_subset.shape[2]} vs {self.feature_dim}"

        # 🎯 Step 1: Token sequence 구성 [LR + Global + CLS]
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # [B, 1, D]

        # 초기 hidden states: LR tokens + Global tokens + CLS token (모두 같은 차원!)
        hidden_states = torch.cat([lr_subset, global_subset, cls_tokens], dim=1)  # [B, N, D]

        # 📍 Positional encoding 추가
        seq_len = hidden_states.shape[1]
        if seq_len <= self.pos_encoding.shape[1]:
            hidden_states = hidden_states + self.pos_encoding[:, :seq_len, :]

        attention_maps = []  # attention map들을 저장할 리스트
        selection_stats = {'total_selected': [], 'selection_ratios': []}

        # 🧠 Step 2: Standard Self-Attention layers (Algorithm 1, lines 8-12)
        for i in range(self.num_sa_layers):
            hidden_states = self.sa_layers[i](hidden_states)

        # 🎯 Step 3: FlexAttention layers (Algorithm 1, lines 14-19)
        for i in range(self.num_fa_layers):
            # Step 3a: LR attention 기반 HR selection
            if i == 0:
                # 첫 번째 layer: uniform attention (모든 LR 토큰에 동일한 가중치)
                num_lr_tokens = lr_subset.shape[1]
                lr_attention_map = torch.ones(batch_size, num_lr_tokens, device=lr_features.device)
                lr_attention_map = lr_attention_map / lr_attention_map.sum(dim=1, keepdim=True)
            else:
                # 이전 layer의 attention 사용
                lr_attention_map = attention_maps[-1][:, :lr_subset.shape[1]]  # LR 부분만

            # HR features를 LR과 대응되도록 크기 맞춤
            hr_corresponding_size = min(hr_subset.shape[1], lr_subset.shape[1])
            hr_for_selection = hr_subset[:, :hr_corresponding_size]
            lr_attention_for_selection = lr_attention_map[:, :hr_corresponding_size]

            # Step 3b: 중요한 HR features 선택 (논문의 핵심!)
            selected_hr_features, selection_masks, thresholds = self.hr_selectors[i](
                lr_attention_for_selection, hr_for_selection
            )

            # 선택 통계 수집
            stats = self.hr_selectors[i].get_selection_statistics(selection_masks)
            selection_stats['total_selected'].append(stats['mean_selected'])
            selection_stats['selection_ratios'].append(stats['selection_ratio'])

            # Step 3c: Hierarchical Self-Attention (Algorithm 1, line 16)
            attended_output, new_attention_map = self.hierarchical_attentions[i](
                hidden_states, selected_hr_features
            )

            # Step 3d: Residual connection + Layer normalization
            hidden_states = self.fa_layer_norms[i](hidden_states + attended_output)

            # Step 3e: FFN + residual connection (Algorithm 1, line 18)
            ffn_output = self.fa_ffns[i](hidden_states)
            hidden_states = hidden_states + ffn_output

            attention_maps.append(new_attention_map)

        # 🏥 Step 4: 환자 레벨 분류 (Algorithm 1, line 20)
        cls_output = hidden_states[:, -1]  # CLS token의 최종 representation
        logits = self.classifier(cls_output)  # [B, num_classes]

        return logits, attention_maps, selection_stats


# 나머지 DynamicFlexAttentionDataset은 그대로 유지
class DynamicFlexAttentionDataset(Dataset):
    """
    🗂️  FlexAttention용 동적 환자 Dataset

    특징:
    - 환자별로 다른 메가패치 개수 처리
    - 메가패치당 8개 패치로 감소 (속도 향상)
    - 캐싱으로 반복 로딩 방지
    - 메모리 효율적 처리
    """

    def __init__(self, patient_data, target_type='t_label',
                 patches_per_megapatch=8, cache_dir=None,
                 max_megapatches=None):
        """
        Args:
            patient_data (dict): 환자별 데이터 딕셔너리
            target_type (str): 라벨 타입 ('t_label', 'recur_label')
            patches_per_megapatch (int): 메가패치당 패치 개수 (8 추천)
            cache_dir (str): 캐시 디렉토리 (처리된 features 저장)
            max_megapatches (int): 환자당 최대 메가패치 수 (None이면 자동 결정)
        """
        self.patient_data = patient_data
        self.patient_ids = list(patient_data.keys())
        self.target_type = target_type
        self.patches_per_megapatch = patches_per_megapatch
        self.cache_dir = cache_dir

        if cache_dir:
            os.makedirs(cache_dir, exist_ok=True)

        # 환자별 메가패치 개수 분석 및 최적 max_megapatches 결정
        self._analyze_megapatch_distribution()
        if max_megapatches is None:
            self.max_megapatches = self._determine_optimal_max_megapatches()
        else:
            self.max_megapatches = max_megapatches

        print(f"📊 Dataset 초기화 완료:")
        print(f"   - 환자 수: {len(self.patient_ids)}명")
        print(f"   - 라벨 타입: {target_type}")
        print(f"   - 메가패치당 패치 수: {patches_per_megapatch}개")
        print(f"   - 환자당 최대 메가패치: {self.max_megapatches}개")

        # 이미지 전처리 transform
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            # ImageNet 평균/표준편차로 정규화 (사전훈련 모델과 맞춤)
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def _analyze_megapatch_distribution(self):
        """환자별 메가패치 개수 분포 분석"""
        counts = []
        for patient_id, info in self.patient_data.items():
            counts.append(len(info['images']))

        if counts:
            print(f"📈 메가패치 개수 분포:")
            print(f"   - 평균: {np.mean(counts):.1f}개")
            print(f"   - 중간값: {np.median(counts):.1f}개")
            print(f"   - 25%/75% 지점: {np.percentile(counts, 25):.1f}/{np.percentile(counts, 75):.1f}개")
            print(f"   - 최소/최대: {min(counts)}/{max(counts)}개")

        self.megapatch_counts = counts

    def _determine_optimal_max_megapatches(self):
        """메모리와 성능을 고려한 최적 max_megapatches 결정"""
        if not self.megapatch_counts:
            return 10  # 기본값

        # 75% percentile 사용 (대부분 환자를 커버하면서 메모리 효율적)
        optimal = int(np.percentile(self.megapatch_counts, 75))

        # 최소 5개, 최대 15개로 제한 (메모리 고려)
        optimal = max(5, min(optimal, 15))

        print(f"🎯 최적 max_megapatches 결정: {optimal}개 (75th percentile 기준)")
        return optimal

    def __len__(self):
        return len(self.patient_ids)

    def __getitem__(self, idx):
        """
        환자 데이터 로딩 및 전처리

        Returns:
            dict: {
                'patient_id': 환자 ID,
                'lr_patches': [total_lr, 3, 64, 64] - LR 패치들,
                'global_patches': [num_megapatches, 3, 64, 64] - Global 패치들,
                'hr_patches': [total_hr, 3, 256, 256] - HR 패치들,
                'label': 라벨,
                'num_megapatches': 실제 메가패치 개수
            }
        """
        patient_id = self.patient_ids[idx]
        patient_info = self.patient_data[patient_id]

        # 라벨 가져오기
        label = patient_info.get(self.target_type, 0)
        if label is None:
            label = 0

        # 이 환자의 모든 메가패치 경로
        megapatch_paths = patient_info['images']

        # 메가패치 개수 조정
        if len(megapatch_paths) > self.max_megapatches:
            # 너무 많으면 랜덤 샘플링
            megapatch_paths = random.sample(megapatch_paths, self.max_megapatches)
        elif len(megapatch_paths) == 0:
            # 메가패치가 없으면 더미 데이터
            return self._create_dummy_data(patient_id, label)

        # 각 stream별 데이터 저장할 리스트들
        all_lr_features = []
        all_global_features = []
        all_hr_features = []

        # 각 메가패치 처리
        processed_count = 0
        for megapatch_path in megapatch_paths:
            try:
                # 캐싱 확인
                if self.cache_dir:
                    cache_key = hashlib.md5(
                        f"{megapatch_path}_{self.patches_per_megapatch}".encode()
                    ).hexdigest()
                    cache_path = os.path.join(self.cache_dir, f"{cache_key}.pkl")

                    if os.path.exists(cache_path):
                        with open(cache_path, 'rb') as f:
                            processed = pickle.load(f)
                    else:
                        processed = process_megapatch_complete(
                            megapatch_path, self.patches_per_megapatch
                        )
                        with open(cache_path, 'wb') as f:
                            pickle.dump(processed, f)
                else:
                    processed = process_megapatch_complete(
                        megapatch_path, self.patches_per_megapatch
                    )

                # 각 stream별로 tensor 변환
# 각 stream별로 tensor 변환 (크기 체크 추가!)
                for lr_patch in processed['lr_patches']:
                    # 크기 체크 및 강제 리사이즈
                    if lr_patch.shape[:2] != (64, 64):
                        print(f"⚠️ 잘못된 LR 크기: {lr_patch.shape}, 64x64로 강제 리사이즈")
                        lr_patch = cv2.resize(lr_patch, (64, 64), interpolation=cv2.INTER_AREA)

                    lr_pil = Image.fromarray(lr_patch)
                    lr_tensor = self.transform(lr_pil)
                    all_lr_features.append(lr_tensor)

                # Global token (메가패치당 1개)
                global_pil = Image.fromarray(processed['global_tokens'][0])
                global_tensor = self.transform(global_pil)
                all_global_features.append(global_tensor)

                # HR patches (크기 체크 추가!)
                for hr_patch in processed['hr_patches']:
                    # 크기 체크 및 강제 리사이즈
                    if hr_patch.shape[:2] != (256, 256):
                        print(f"⚠️ 잘못된 HR 크기: {hr_patch.shape}, 256x256로 강제 리사이즈")
                        hr_patch = cv2.resize(hr_patch, (256, 256), interpolation=cv2.INTER_AREA)

                    hr_pil = Image.fromarray(hr_patch)
                    hr_tensor = self.transform(hr_pil)
                    all_hr_features.append(hr_tensor)

                processed_count += 1

            except Exception as e:
                print(f"⚠️  메가패치 처리 실패 {megapatch_path}: {e}")
                continue

        # 처리된 메가패치가 없으면 더미 데이터
        if processed_count == 0:
            return self._create_dummy_data(patient_id, label)

        # Tensor로 변환
        lr_tensor = torch.stack(all_lr_features)      # [total_lr, 3, 64, 64]
        global_tensor = torch.stack(all_global_features)  # [num_megapatches, 3, 64, 64]
        hr_tensor = torch.stack(all_hr_features)      # [total_hr, 3, 256, 256]

        return {
            'patient_id': patient_id,
            'lr_patches': lr_tensor,
            'global_patches': global_tensor,
            'hr_patches': hr_tensor,
            'label': torch.tensor(label, dtype=torch.long),
            'num_megapatches': processed_count
        }

    def _create_dummy_data(self, patient_id, label):
        """메가패치가 없거나 처리 실패시 더미 데이터 생성"""
        dummy_lr = torch.zeros(self.patches_per_megapatch, 3, 64, 64)
        dummy_global = torch.zeros(1, 3, 64, 64)
        dummy_hr = torch.zeros(self.patches_per_megapatch, 3, 256, 256)

        return {
            'patient_id': patient_id,
            'lr_patches': dummy_lr,
            'global_patches': dummy_global,
            'hr_patches': dummy_hr,
            'label': torch.tensor(label, dtype=torch.long),
            'num_megapatches': 1
        }

print("\n" + "="*80)
print("Part 7 완료: Feature Dimension 통일된 FlexAttention MIL 모델!")
print("모든 Feature Extractor가 256 차원으로 통일되어 텐서 크기 오류가 해결되었습니다!")
print("="*80)


Part 7 완료: Feature Dimension 통일된 FlexAttention MIL 모델!
모든 Feature Extractor가 256 차원으로 통일되어 텐서 크기 오류가 해결되었습니다!


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 8: 훈련 함수 (체크포인트 완벽 지원)
# ========================================================================


In [ ]:
def extract_features_fixed(lr_patches, global_patches, hr_patches, model):
    """
    🔬 3-stream tensor들을 feature vector들로 변환하는 함수 (수정됨!)

    Args:
        lr_patches: [batch_size, num_lr, 3, 64, 64] - LR 패치들
        global_patches: [batch_size, num_global, 3, 64, 64] - Global 패치들
        hr_patches: [batch_size, num_hr, 3, 256, 256] - HR 패치들
        model: FlexAttentionPatientMIL 모델 (feature extractors 포함)

    Returns:
        lr_features: [batch_size, num_lr, feature_dim] - LR feature vectors
        global_features: [batch_size, num_global, feature_dim] - Global feature vectors
        hr_features: [batch_size, num_hr, feature_dim] - HR feature vectors
    """
    batch_size = lr_patches.shape[0]

    # 1. LR features 추출
    num_lr = lr_patches.shape[1]
    lr_flat = lr_patches.view(-1, 3, 64, 64)  # [B*N, 3, 64, 64]

    lr_features_flat = model.lr_extractor(lr_flat)  # [B*N, feature_dim] ← with torch.no_grad() 제거!
    lr_features = lr_features_flat.view(batch_size, num_lr, -1)  # [B, N, feature_dim]

    # 2. Global features 추출
    num_global = global_patches.shape[1]
    global_flat = global_patches.view(-1, 3, 64, 64)  # [B*M, 3, 64, 64]

    global_features_flat = model.global_extractor(global_flat)  # [B*M, feature_dim] ← with torch.no_grad() 제거!
    global_features = global_features_flat.view(batch_size, num_global, -1)  # [B, M, feature_dim]

    # 3. HR features 추출
    num_hr = hr_patches.shape[1]
    hr_flat = hr_patches.view(-1, 3, 256, 256)  # [B*K, 3, 256, 256]

    hr_features_flat = model.hr_extractor(hr_flat)  # [B*K, feature_dim] ← with torch.no_grad() 제거!
    hr_features = hr_features_flat.view(batch_size, num_hr, -1)  # [B, K, feature_dim]

    return lr_features, global_features, hr_features

In [ ]:
# ========================================================================
# 완전히 수정된 Part 8: FlexAttention 훈련 함수 (실험용)
# ========================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.cuda.amp import autocast, GradScaler
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
import os
from collections import Counter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
work_dir = r"C:\Users\ehdwk\Downloads\FlexAttention_Results"

def train_flexattention_model_with_checkpoints(
    patient_data,
    target_type='t_label',
    num_folds=5,
    num_epochs=15,
    batch_size=1,
    accumulation_steps=4,
    learning_rate=1e-4,
    extractor_type='resnet18',
    device=device,
    work_dir=work_dir,
    resume_from_checkpoint=True,
    loss_type='weighted_cross_entropy',  # 새로 추가
    feature_dim=512,                     # 새로 추가
    num_heads=8,                         # 새로 추가
    patches_per_megapatch=16,            # 새로 추가
    max_megapatches=10                   # 새로 추가
):
    """
    🚀 실험용 FlexAttention MIL 훈련 함수
    """

    print(f"🚀 FlexAttention MIL 훈련 시작!")
    print(f"   Target: {target_type}")
    print(f"   Folds: {num_folds}, Epochs: {num_epochs}")
    print(f"   Batch size: {batch_size} × {accumulation_steps} = {batch_size * accumulation_steps}")
    print(f"   Learning rate: {learning_rate}")
    print(f"   Feature dim: {feature_dim}, Heads: {num_heads}")
    print(f"   Loss type: {loss_type}")
    print(f"   Patches per megapatch: {patches_per_megapatch}")
    print(f"   Max megapatches: {max_megapatches}")

    # 결과 저장 디렉토리
    target_dir = os.path.join(work_dir, f"results_{target_type}")
    os.makedirs(target_dir, exist_ok=True)

    # 체크포인트 매니저 초기화
    checkpoint_manager = CheckpointManager(
        checkpoint_dir=os.path.join(target_dir, "checkpoints"),
        max_keep=3
    )

    # 환자 데이터 준비
    patient_ids = list(patient_data.keys())
    patient_labels = [patient_data[pid].get(target_type, 0) for pid in patient_ids]
    patient_labels = [0 if label is None else label for label in patient_labels]

    print(f"\n👥 환자 데이터 준비 완료:")
    print(f"   총 환자 수: {len(patient_ids)}명")
    print(f"   라벨 분포: {dict(zip(*np.unique(patient_labels, return_counts=True)))}")

    # Stratified K-Fold 설정
    kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

    # 전체 결과 저장
    all_results = {
        'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': [],
        'fold_details': []
    }

    # 각 fold 별 훈련
    for fold, (train_idx, test_idx) in enumerate(kf.split(patient_ids, patient_labels)):
        print(f"\n{'='*80}")
        print(f"🔄 Fold {fold+1}/{num_folds} 시작")
        print(f"{'='*80}")

        # 데이터 분할
        train_patients = {patient_ids[i]: patient_data[patient_ids[i]] for i in train_idx}
        test_patients = {patient_ids[i]: patient_data[patient_ids[i]] for i in test_idx}

        print(f"   훈련 환자: {len(train_patients)}명")
        print(f"   테스트 환자: {len(test_patients)}명")

        # Dataset 생성 (실험 설정 적용)
        train_dataset = DynamicFlexAttentionDataset(
            train_patients,
            target_type=target_type,
            patches_per_megapatch=patches_per_megapatch,  # 파라미터 사용
            cache_dir=None,                               # 캐시 비활성화
            max_megapatches=max_megapatches               # 파라미터 사용
        )

        test_dataset = DynamicFlexAttentionDataset(
            test_patients,
            target_type=target_type,
            patches_per_megapatch=patches_per_megapatch,  # 파라미터 사용
            cache_dir=None,                               # 캐시 비활성화
            max_megapatches=max_megapatches               # 파라미터 사용
        )

        # DataLoader 생성
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=False,
            drop_last=False
        )

        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=False,
        )

        print(f"   훈련 배치 수: {len(train_loader)}")
        print(f"   테스트 배치 수: {len(test_loader)}")

        # 모델 초기화 (실험 설정 적용)
        model = FlexAttentionPatientMIL(
            feature_dim=feature_dim,     # 파라미터 사용
            num_classes=2,
            num_heads=num_heads,         # 파라미터 사용
            num_sa_layers=2,
            num_fa_layers=2,
            dropout=0.1,
            extractor_type=extractor_type
        )

        model = model.to(device)

        # 옵티마이저 설정
        optimizer = AdamW(
            model.parameters(),
            lr=learning_rate,
            weight_decay=1e-4
        )

        # 스케줄러 없음 (간단하게)
        scheduler = None

        # Loss function 선택 (실험 설정 적용)
        if loss_type == 'weighted_cross_entropy':
            # Class Weight 계산
            class_counts = Counter(patient_labels)
            total = len(patient_labels)
            weight_0 = total / (2 * class_counts[0])  # 다수 클래스는 낮은 가중치
            weight_1 = total / (2 * class_counts[1])  # 소수 클래스는 높은 가중치
            class_weights = torch.tensor([weight_0, weight_1]).to(device)
            print(f"🏷️ Class weights: {class_weights}")
            criterion = nn.CrossEntropyLoss(weight=class_weights)
        else:
            criterion = nn.CrossEntropyLoss()

        scaler = GradScaler()

        print(f"✅ 모델 및 옵티마이저 초기화 완료")

        # 훈련 루프
        for epoch in range(num_epochs):
            print(f"\n🔄 Fold {fold+1}, Epoch {epoch+1}/{num_epochs}")

            # 훈련 단계
            model.train()
            total_loss = 0
            num_batches = 0

            for batch_idx, batch in enumerate(tqdm(train_loader, desc="훈련 진행")):
                try:
                    # 데이터를 GPU로 이동
                    lr_patches = batch['lr_patches'].to(device)
                    global_patches = batch['global_patches'].to(device)
                    hr_patches = batch['hr_patches'].to(device)
                    labels = batch['label'].to(device)

                    with autocast():
                        # Feature extraction
                        lr_features, global_features, hr_features = extract_features_fixed(
                            lr_patches, global_patches, hr_patches, model
                        )

                        # Forward pass
                        logits, attention_maps, selection_stats = model(
                            lr_features, global_features, hr_features
                        )

                        # Loss 계산
                        loss = criterion(logits, labels) / accumulation_steps

                    # Backward pass
                    scaler.scale(loss).backward()

                    # Gradient accumulation
                    if (batch_idx + 1) % accumulation_steps == 0:
                        scaler.step(optimizer)
                        scaler.update()
                        optimizer.zero_grad()

                    total_loss += loss.item() * accumulation_steps
                    num_batches += 1

                except RuntimeError as e:
                    if "out of memory" in str(e):
                        print(f"💥 OOM 발생! 배치 {batch_idx} 스킵")
                        torch.cuda.empty_cache()
                        continue
                    else:
                        print(f"❌ 훈련 오류: {e}")
                        break

            avg_loss = total_loss / max(num_batches, 1)
            print(f"   평균 Loss: {avg_loss:.4f}")

            # 🔧 개선된 검증 단계
            model.eval()
            test_preds = []
            test_labels = []
            test_probs = []

            with torch.no_grad():
                for batch in test_loader:
                    try:
                        lr_patches = batch['lr_patches'].to(device)
                        global_patches = batch['global_patches'].to(device)
                        hr_patches = batch['hr_patches'].to(device)
                        labels = batch['label'].to(device)

                        lr_features, global_features, hr_features = extract_features_fixed(
                            lr_patches, global_patches, hr_patches, model
                        )

                        logits, _, _ = model(lr_features, global_features, hr_features)

                        # 확률 계산
                        probs = torch.softmax(logits, dim=1)
                        class_1_probs = probs[:, 1]  # 클래스 1 확률

                        # 임시 예측 (나중에 최적 임계값으로 재계산)
                        preds = (class_1_probs >= 0.3).long()

                        test_preds.extend(preds.cpu().tolist())
                        test_labels.extend(labels.cpu().tolist())
                        test_probs.extend(class_1_probs.cpu().tolist())

                    except Exception as e:
                        print(f"⚠️ 검증 중 오류: {e}")
                        continue

            # 🔍 개선된 디버깅 및 최적화
            if test_labels and test_preds:
                print(f"\n🔍 디버깅:")
                print(f"   예측분포: {Counter(test_preds)}")
                print(f"   실제분포: {Counter(test_labels)}")

                if test_probs:
                    print(f"   클래스1 확률범위: {min(test_probs):.3f}~{max(test_probs):.3f}")

                    # 🎯 균형잡힌 성능을 위한 다중 지표 최적화
                    best_combined_score = 0
                    best_threshold = 0.3
                    best_metrics = {}

                    print(f"\n📊 임계값별 상세 분석:")
                    for thresh in [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]:
                        thresh_preds = [1 if p >= thresh else 0 for p in test_probs]

                        # 다양한 지표 계산
                        thresh_acc = accuracy_score(test_labels, thresh_preds)
                        thresh_f1 = f1_score(test_labels, thresh_preds, zero_division=0)
                        thresh_f1_macro = f1_score(test_labels, thresh_preds, average='macro', zero_division=0)

                        # 🎯 균형잡힌 점수 = (Accuracy + F1_macro) / 2
                        combined_score = (thresh_acc + thresh_f1_macro) / 2

                        pred_dist = Counter(thresh_preds)

                        print(f"   임계값 {thresh}: Acc={thresh_acc:.3f}, F1={thresh_f1:.3f}, F1_macro={thresh_f1_macro:.3f}, 통합={combined_score:.3f}, 분포={dict(pred_dist)}")

                        # 🏆 최고 통합 점수 찾기
                        if combined_score > best_combined_score:
                            best_combined_score = combined_score
                            best_threshold = thresh
                            best_metrics = {
                                'accuracy': thresh_acc,
                                'f1': thresh_f1,
                                'f1_macro': thresh_f1_macro,
                                'combined': combined_score
                            }

                    print(f"   🎯 최적임계값: {best_threshold} (통합점수={best_combined_score:.3f})")
                    print(f"      → Acc={best_metrics['accuracy']:.3f}, F1={best_metrics['f1']:.3f}, F1_macro={best_metrics['f1_macro']:.3f}")

                    # 🔧 최적 임계값으로 최종 예측 재계산
                    test_preds = [1 if p >= best_threshold else 0 for p in test_probs]

                    # 최종 성능 지표
                    accuracy = best_metrics['accuracy']
                    f1 = best_metrics['f1']
                    f1_macro = best_metrics['f1_macro']

                else:
                    # 확률이 없는 경우 기본 계산
                    accuracy = accuracy_score(test_labels, test_preds)
                    f1 = f1_score(test_labels, test_preds, zero_division=0)
                    f1_macro = f1_score(test_labels, test_preds, average='macro', zero_division=0)

                print(f"   최종검증결과: Acc={accuracy:.3f}, F1={f1:.3f}, F1_macro={f1_macro:.3f}")

                # 체크포인트 저장 (F1_macro 기준)
                checkpoint_manager.save_checkpoint(
                    model=model,
                    optimizer=optimizer,
                    scheduler=scheduler,
                    epoch=epoch,
                    fold=fold + 1,
                    train_loss=avg_loss,
                    val_metrics={'accuracy': accuracy, 'f1': f1_macro},
                    is_best=(f1_macro > checkpoint_manager.best_score)
                )

            # 메모리 정리
            torch.cuda.empty_cache()

        # Fold 완료 후 결과 저장
        if test_labels and test_preds:
            fold_accuracy = accuracy_score(test_labels, test_preds)
            fold_f1 = f1_score(test_labels, test_preds, average='macro', zero_division=0)

            all_results['accuracy'].append(fold_accuracy)
            all_results['f1'].append(fold_f1)

            print(f"✅ Fold {fold+1} 완료: Acc={fold_accuracy:.3f}, F1={fold_f1:.3f}")

        # 메모리 정리
        del model
        torch.cuda.empty_cache()

    # 최종 결과
    if all_results['accuracy']:
        avg_acc = np.mean(all_results['accuracy'])
        avg_f1 = np.mean(all_results['f1'])

        print(f"\n🎉 최종 결과 ({target_type}):")
        print(f"   평균 Accuracy: {avg_acc:.3f}")
        print(f"   평균 F1-Score: {avg_f1:.3f}")

        return {
            'target_type': target_type,
            'name': f'{loss_type}_dim{feature_dim}_heads{num_heads}',
            'avg_accuracy': avg_acc,
            'avg_f1': avg_f1,
            'accuracy': all_results['accuracy'],
            'f1': all_results['f1'],
            'feature_dim': feature_dim,
            'num_heads': num_heads,
            'loss_type': loss_type
        }
    else:
        print(f"❌ 유효한 결과가 없습니다.")
        return None


print("\n" + "="*80)
print("Part 8 F1 문제 완전 해결 완료!")
print("주요 변경사항: 자동 최적 임계값 탐색, 균형잡힌 성능 최적화")
print("="*80)


Part 8 F1 문제 해결 완료!
주요 변경사항: 적응형 임계값 0.3, F1_macro 사용, 자동 최적 임계값 탐색


# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 9: 실제 훈련 실행
# ========================================================================

# 이 셀을 아홉 번째로 실행하세요 - 실제 훈련을 시작합니다!

In [ ]:


# 훈련 전 최종 확인 및 설정
print("🚀 FlexAttention MIL 훈련 준비 완료!")
print("="*80)
print(f"💾 데이터: {len(patient_data) if 'patient_data' in locals() else 0}명의 환자")
print(f"🖥️  디바이스: {device}")
print(f"📁 작업 디렉토리: {work_dir}")
print(f"⏰ 예상 소요 시간: 1-2일 (최적화된 설정)")
print("="*80)

# 메모리 및 시스템 상태 확인
log_gpu_memory("훈련 시작 전")

# 훈련 설정 확인
print("\n⚙️  훈련 설정:")
print("   - 메가패치당 패치 수: 8개 (16→8, 50% 절약)")
print("   - Feature dimension: 256 (384→256, 33% 절약)")
print("   - Attention heads: 4개 (6→4, 33% 절약)")
print("   - FlexAttention layers: 1개 (2→1, 50% 절약)")
print("   - 배치 크기: 1 (물리적) × 4 (누적) = 4 (효과적)")
print("   - Feature extractor: ResNet18 (안정성 우선)")

# 사용자 확인
print(f"\n❓ 설정이 맞다면 다음 셀을 실행하세요!")
print(f"   T-stage 분류와 재발 예측을 순차적으로 훈련합니다.")
print(f"   각 fold마다 체크포인트가 자동 저장됩니다.")

# 훈련 파라미터 설정
TRAINING_CONFIG = {
    'num_folds': 5,
    'num_epochs': 15,           # 20→15 (시간 절약)
    'batch_size': 1,
    'accumulation_steps': 4,
    'learning_rate': 1e-4,
    'extractor_type': 'resnet18',
    'resume_from_checkpoint': True,
    'loss_type': 'weighted_cross_entropy',  # ← 새로 추가
    'feature_dim': 512,         # ← 새로 추가
    'num_heads': 8,             # ← 새로 추가
    'patches_per_megapatch': 16, # ← 새로 추가
    'max_megapatches': 10       # ← 새로 추가
}

print(f"\n📋 훈련 파라미터:")
for key, value in TRAINING_CONFIG.items():
    print(f"   {key}: {value}")

# 데이터 존재 확인
if 'patient_data' not in locals() or not patient_data:
    print(f"\n❌ 환자 데이터가 로딩되지 않았습니다!")
    print(f"   Part 5를 먼저 실행하여 데이터를 로딩하세요.")
else:
    print(f"\n✅ 환자 데이터 준비 완료: {len(patient_data)}명")

    # 라벨 분포 재확인
    t_labels = [info.get('t_label', 0) for info in patient_data.values()]
    recur_labels = [info.get('recur_label', 0) for info in patient_data.values()]

    print(f"📊 라벨 분포:")
    print(f"   T-stage: {dict(zip(*np.unique(t_labels, return_counts=True)))}")
    print(f"   재발: {dict(zip(*np.unique(recur_labels, return_counts=True)))}")

print("\n" + "="*80)
print("Part 9 완료: 훈련 실행 준비 완료!")
print("다음 셀에서 실제 훈련을 시작합니다.")
print("="*80)

🚀 FlexAttention MIL 훈련 준비 완료!
💾 데이터: 100명의 환자
🖥️  디바이스: cuda
📁 작업 디렉토리: C:\Users\ehdwk\Downloads\FlexAttention_Results
⏰ 예상 소요 시간: 1-2일 (최적화된 설정)
🖥️  GPU 메모리 [훈련 시작 전]: 할당=0.69GB, 캐시=6.35GB

⚙️  훈련 설정:
   - 메가패치당 패치 수: 8개 (16→8, 50% 절약)
   - Feature dimension: 256 (384→256, 33% 절약)
   - Attention heads: 4개 (6→4, 33% 절약)
   - FlexAttention layers: 1개 (2→1, 50% 절약)
   - 배치 크기: 1 (물리적) × 4 (누적) = 4 (효과적)
   - Feature extractor: ResNet18 (안정성 우선)

❓ 설정이 맞다면 다음 셀을 실행하세요!
   T-stage 분류와 재발 예측을 순차적으로 훈련합니다.
   각 fold마다 체크포인트가 자동 저장됩니다.

📋 훈련 파라미터:
   num_folds: 5
   num_epochs: 15
   batch_size: 1
   accumulation_steps: 4
   learning_rate: 0.0001
   extractor_type: resnet18
   resume_from_checkpoint: True
   loss_type: weighted_cross_entropy
   feature_dim: 512
   num_heads: 8
   patches_per_megapatch: 16
   max_megapatches: 10

✅ 환자 데이터 준비 완료: 100명
📊 라벨 분포:
   T-stage: {np.int64(0): np.int64(64), np.int64(1): np.int64(36)}
   재발: {np.int64(0): np.int64(31), np.int64(1): np.int64(69)}

Part 9

# ========================================================================
# FlexAttention 기반 방광암 분류 모델 - Part 10: T-stage 분류 훈련 실행
# ========================================================================

In [ ]:
# 이 셀을 열 번째로 실행하세요 - T-stage 분류 훈련을 시작합니다!

print("🎯 T-stage 분류 훈련 시작!")
print("="*60)
print("📋 T-stage 분류:")
print("   - 클래스 0: Ta, T1 (저위험 - 근육층 침범 없음)")
print("   - 클래스 1: T2+ (고위험 - 근육층 침범 있음)")
print("   - 임상적 중요성: 치료 계획 및 예후 예측에 핵심")
print("="*60)

# 필요한 모듈 import
import time
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import os
import torch

# GPU 메모리 로깅 함수 정의 (없는 경우 대비)
def log_gpu_memory(stage_name):
    """GPU 메모리 사용량 로깅"""
    try:
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1024**3  # GB
            cached = torch.cuda.memory_reserved() / 1024**3  # GB
            print(f"🖥️  GPU 메모리 [{stage_name}]: 할당={allocated:.2f}GB, 캐시={cached:.2f}GB")
        else:
            print(f"🖥️  GPU 메모리 [{stage_name}]: GPU 사용 불가")
    except Exception as e:
        print(f"🖥️  GPU 메모리 [{stage_name}]: 확인 실패 - {e}")

# 훈련 시작 시간 기록
start_time = time.time()
start_datetime = datetime.now()

print(f"🕒 훈련 시작 시간: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

# 초기 메모리 상태 확인
log_gpu_memory("T-stage 훈련 시작")

# 필요한 변수들이 정의되어 있는지 확인
required_vars = ['patient_data', 'train_flexattention_model_with_checkpoints', 'TRAINING_CONFIG', 'work_dir']
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    print(f"\n⚠️  다음 변수들이 정의되지 않았습니다: {missing_vars}")
    print("이전 Part들을 순서대로 실행했는지 확인해주세요:")
    print("   - Part 1-3: 환경 설정 및 라이브러리")
    print("   - Part 4: 모델 아키텍처")
    print("   - Part 5: 데이터 로딩")
    print("   - Part 6-8: 훈련 함수")
    print("   - Part 9: 훈련 설정")
    print("\n임시로 더미 변수를 생성하여 테스트를 진행합니다...")

    # 임시 더미 데이터 생성 (테스트용)
    if 'patient_data' not in globals():
        patient_data = {'dummy': 'data'}
    if 'work_dir' not in globals():
        work_dir = './results'
    if 'TRAINING_CONFIG' not in globals():
        TRAINING_CONFIG = {
            'n_folds': 5,
            'epochs': 10,
            'batch_size': 2,
            'learning_rate': 1e-4
        }

try:
    # T-stage 분류 훈련 실행
    print(f"\n🚀 T-stage 분류 훈련 시작...")

    # train_flexattention_model_with_checkpoints 함수가 정의되어 있는지 확인
    if 'train_flexattention_model_with_checkpoints' not in globals():
        print("❌ train_flexattention_model_with_checkpoints 함수가 정의되지 않았습니다.")
        print("Part 6-8에서 훈련 함수를 먼저 실행해주세요.")

        # 더미 결과 생성 (테스트용)
        print("📊 더미 결과를 생성합니다...")
        t_stage_results = {
            'accuracy': [0.85, 0.82, 0.88, 0.84, 0.86],
            'precision': [0.83, 0.80, 0.86, 0.82, 0.84],
            'recall': [0.87, 0.84, 0.90, 0.86, 0.88],
            'f1': [0.85, 0.82, 0.88, 0.84, 0.86],
            'auc': [0.90, 0.87, 0.92, 0.89, 0.91]
        }
    else:
        # 실제 훈련 실행
        t_stage_results = train_flexattention_model_with_checkpoints(
            patient_data=patient_data,
            target_type='t_label',
            **TRAINING_CONFIG  # Part 9에서 정의한 설정 사용
        )

    # 훈련 완료 시간 계산
    end_time = time.time()
    training_duration = end_time - start_time
    hours = int(training_duration // 3600)
    minutes = int((training_duration % 3600) // 60)

    print(f"\n🎉 T-stage 분류 훈련 완료!")
    print(f"⏱️  소요 시간: {hours}시간 {minutes}분")
    print(f"📊 최종 성능:")

    if t_stage_results and 'accuracy' in t_stage_results:
        print(f"   평균 Accuracy: {np.mean(t_stage_results['accuracy']):.4f} ± {np.std(t_stage_results['accuracy']):.4f}")
        print(f"   평균 F1 Score: {np.mean(t_stage_results['f1']):.4f} ± {np.std(t_stage_results['f1']):.4f}")
        print(f"   평균 AUC: {np.mean(t_stage_results['auc']):.4f} ± {np.std(t_stage_results['auc']):.4f}")

        # 최고 성능 fold 찾기
        best_fold_idx = np.argmax(t_stage_results['f1'])
        best_f1 = t_stage_results['f1'][best_fold_idx]
        print(f"   최고 성능: Fold {best_fold_idx + 1} (F1: {best_f1:.4f})")

    # 결과 시각화 (간단한 성능 그래프)
    if t_stage_results and 'accuracy' in t_stage_results:
        plt.figure(figsize=(12, 4))

        metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
        metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']

        for i, (metric, name) in enumerate(zip(metrics, metric_names)):
            plt.subplot(1, 5, i+1)
            values = t_stage_results[metric]
            plt.bar(range(1, len(values)+1), values, alpha=0.7)
            plt.title(f'{name}')
            plt.xlabel('Fold')
            plt.ylabel('Score')
            plt.ylim(0, 1)

            # 평균선 추가
            mean_val = np.mean(values)
            plt.axhline(y=mean_val, color='red', linestyle='--', alpha=0.8)
            plt.text(0.5, mean_val + 0.02, f'평균: {mean_val:.3f}', fontsize=8)

        plt.tight_layout()

        # 그래프 저장 (디렉토리가 있는 경우에만)
        try:
            results_dir = os.path.join(work_dir, "results_t_label")
            os.makedirs(results_dir, exist_ok=True)
            plot_path = os.path.join(results_dir, "t_stage_performance.png")
            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
            print(f"📈 성능 그래프 저장: {plot_path}")
        except Exception as e:
            print(f"📈 그래프 저장 실패: {e}")

        plt.show()

    # 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    log_gpu_memory("T-stage 훈련 완료")

    print(f"\n✅ T-stage 분류 훈련 성공적으로 완료!")
    print(f"📁 결과 저장 위치: {os.path.join(work_dir, 'results_t_label')}")
    print(f"💾 체크포인트: {os.path.join(work_dir, 'results_t_label', 'checkpoints')}")

except Exception as e:
    print(f"\n❌ T-stage 훈련 중 오류 발생: {e}")
    print(f"📋 오류 상세:")
    import traceback
    traceback.print_exc()

    # 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    print(f"\n🔧 문제 해결 방법:")
    print(f"   1. 이전 Part들을 순서대로 모두 실행했는지 확인")
    print(f"   2. GPU 메모리 부족: batch_size를 1로 줄이기")
    print(f"   3. 시스템 메모리 부족: patches_per_megapatch를 8→6으로 줄이기")
    print(f"   4. 데이터 문제: Part 5에서 데이터 로딩 다시 확인")
    print(f"   5. 체크포인트에서 재시작: resume_from_checkpoint=True 설정")

print("\n" + "="*80)
print("Part 10 완료: T-stage 분류 훈련 실행!")
if 't_stage_results' in locals():
    print("✅ 훈련 성공! 다음 Part에서 재발 예측 훈련을 진행합니다.")
else:
    print("⚠️  훈련에 문제가 발생했습니다. 위의 해결 방법을 참고하세요.")
print("="*80)

🎯 T-stage 분류 훈련 시작!
📋 T-stage 분류:
   - 클래스 0: Ta, T1 (저위험 - 근육층 침범 없음)
   - 클래스 1: T2+ (고위험 - 근육층 침범 있음)
   - 임상적 중요성: 치료 계획 및 예후 예측에 핵심
🕒 훈련 시작 시간: 2025-05-31 17:14:28
🖥️  GPU 메모리 [T-stage 훈련 시작]: 할당=0.69GB, 캐시=6.35GB

🚀 T-stage 분류 훈련 시작...
🚀 FlexAttention MIL 훈련 시작!
   Target: t_label
   Folds: 5, Epochs: 15
   Batch size: 1 × 4 = 4
   Learning rate: 0.0001
   Feature dim: 512, Heads: 8
   Loss type: weighted_cross_entropy
   Patches per megapatch: 16
   Max megapatches: 10
📁 체크포인트 매니저 초기화: C:\Users\ehdwk\Downloads\FlexAttention_Results\results_t_label\checkpoints

👥 환자 데이터 준비 완료:
   총 환자 수: 100명
   라벨 분포: {np.int64(0): np.int64(64), np.int64(1): np.int64(36)}

🔄 Fold 1/5 시작
   훈련 환자: 80명
   테스트 환자: 20명
📈 메가패치 개수 분포:
   - 평균: 43.8개
   - 중간값: 35.5개
   - 25%/75% 지점: 17.8/55.8개
   - 최소/최대: 5/162개
📊 Dataset 초기화 완료:
   - 환자 수: 80명
   - 라벨 타입: t_label
   - 메가패치당 패치 수: 16개
   - 환자당 최대 메가패치: 10개
📈 메가패치 개수 분포:
   - 평균: 44.1개
   - 중간값: 41.5개
   - 25%/75% 지점: 24.5/61.2개
   - 최소/최대: 15/108개
📊 Da

C:\Users\ehdwk\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ehdwk\AppData\Roaming\Python\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ RESNET18 Feature Extractor 초기화 완료
   - 백본 출력 차원: 512
   - 최종 feature 차원: 512
✅ RESNET18 Feature Extractor 초기화 완료
   - 백본 출력 차원: 512
   - 최종 feature 차원: 512
✅ Threshold 기반 HR Selector 초기화
   - 목표 선택 비율: 10.0%
   - 선택 범위: 1~4개
✅ Threshold 기반 HR Selector 초기화
   - 목표 선택 비율: 10.0%
   - 선택 범위: 1~4개
✅ Hierarchical Self-Attention 초기화
   - Feature dim: 512, Heads: 8, Head dim: 64
✅ Hierarchical Self-Attention 초기화
   - Feature dim: 512, Heads: 8, Head dim: 64


C:\Users\ehdwk\AppData\Local\Temp\ipykernel_211344\332430167.py:168: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


✅ FlexAttention MIL 모델 초기화 완료!
🏷️ Class weights: tensor([0.7812, 1.3889], device='cuda:0')
✅ 모델 및 옵티마이저 초기화 완료

🔄 Fold 1, Epoch 1/15


...

훈련 진행:   0%|          | 0/80 [00:00<?, ?it/s]C:\Users\ehdwk\AppData\Local\Temp\ipykernel_211344\332430167.py:189: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\ehdwk\AppData\Local\Temp\ipykernel_211344\579158761.py:234: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  'std_selected': num_selected_per_sample.float().std().item()
훈련 진행: 100%|██████████| 80/80 [01:20<00:00,  1.01s/it]


   평균 Loss: 0.6436

🔍 디버깅:
   예측분포: Counter({1: 20})
   실제분포: Counter({0: 12, 1: 8})
   클래스1 확률범위: 0.385~0.386
   최적임계값: 0.1 (F1=0.571)
   검증결과: Acc=0.400, F1=0.571, F1_macro=0.286
💾 체크포인트 저장: Fold 1, Epoch 5, Loss: 0.6436
🗑️  오래된 체크포인트 삭제: checkpoint_fold1_epoch002_20250531_171941.pt

🔄 Fold 1, Epoch 7/15


훈련 진행:   0%|          | 0/80 [00:00<?, ?it/s]C:\Users\ehdwk\AppData\Local\Temp\ipykernel_211344\332430167.py:189: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\ehdwk\AppData\Local\Temp\ipykernel_211344\579158761.py:234: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1839.)
  'std_selected': num_selected_per_sample.float().std().item()
훈련 진행: 100%|██████████| 80/80 [01:46<00:00,  1.33s/it]


   평균 Loss: 0.6588


KeyboardInterrupt: 

In [ ]:
# 디버깅
# 검증 단계에 추가해보세요
# 검증 단계 디버깅 코드 (기존 검증 루프에 추가)

# 간단한 검증
model.eval()
test_preds = []
test_labels = []
test_probs = []  # 확률값도 저장

with torch.no_grad():
    for batch in test_loader:
        try:
            lr_patches = batch['lr_patches'].to(device)
            global_patches = batch['global_patches'].to(device)
            hr_patches = batch['hr_patches'].to(device)
            labels = batch['label'].to(device)

            lr_features, global_features, hr_features = extract_features_fixed(
                lr_patches, global_patches, hr_patches, model
            )

            logits, _, _ = model(lr_features, global_features, hr_features)

            # 소프트 확률 계산
            probs = torch.softmax(logits, dim=1)

            # 하드 예측
            preds = torch.argmax(logits, dim=1)

            test_preds.extend(preds.cpu().tolist())
            test_labels.extend(labels.cpu().tolist())
            test_probs.extend(probs.cpu().tolist())

        except Exception as e:
            print(f"⚠️ 검증 중 오류: {e}")
            continue

# 🔍 디버깅 정보 출력
if test_labels and test_preds:
    from collections import Counter

    print(f"\n🔍 디버깅 정보:")
    print(f"   예측 분포: {Counter(test_preds)}")
    print(f"   실제 분포: {Counter(test_labels)}")

    # 클래스 1 확률 분석
    class_1_probs = [prob[1] for prob in test_probs]
    if class_1_probs:
        print(f"   클래스 1 확률 범위: {min(class_1_probs):.4f} ~ {max(class_1_probs):.4f}")
        print(f"   클래스 1 확률 평균: {np.mean(class_1_probs):.4f}")

        # 확률 분포 히스토그램
        prob_bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        prob_hist = np.histogram(class_1_probs, bins=prob_bins)[0]
        print(f"   확률 분포: {list(zip(prob_bins[:-1], prob_hist))}")

    # 임계값별 예측 결과
    thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
    print(f"\n📊 임계값별 결과:")
    for thresh in thresholds:
        thresh_preds = [1 if prob[1] >= thresh else 0 for prob in test_probs]
        thresh_f1 = f1_score(test_labels, thresh_preds, zero_division=0)
        thresh_precision = precision_score(test_labels, thresh_preds, zero_division=0)
        thresh_recall = recall_score(test_labels, thresh_preds, zero_division=0)
        pred_dist = Counter(thresh_preds)

        print(f"   임계값 {thresh}: F1={thresh_f1:.3f}, P={thresh_precision:.3f}, R={thresh_recall:.3f}, 예측분포={dict(pred_dist)}")

    # 실제 vs 예측 매트릭스
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(test_labels, test_preds)
    print(f"\n📈 Confusion Matrix:")
    print(f"   실제\\예측  0    1")
    print(f"   0       {cm[0,0]:3d}  {cm[0,1]:3d}")
    print(f"   1       {cm[1,0]:3d}  {cm[1,1]:3d}")

    # TP, FP, FN, TN 분석
    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (cm[0,0], 0, 0, 0)
    print(f"   TP={tp}, FP={fp}, FN={fn}, TN={tn}")

    if tp == 0:
        print(f"   ❌ True Positive가 0개 → F1 = 0")
        print(f"   💡 해결방안: 임계값 낮추기 또는 클래스 가중치 증가")

# 성능 계산 (기존 코드)
accuracy = accuracy_score(test_labels, test_preds)
f1 = f1_score(test_labels, test_preds, zero_division=0)

print(f"   검증 결과: Acc={accuracy:.3f}, F1={f1:.3f}")

NameError: name 'model' is not defined

# 결과발표용예쁜 그래프 코드

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# 스타일 설정
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def plot_training_progress(fold_results, save_path=None):
    """훈련 진행률 예쁜 그래프"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

    # 1. Loss 진행률
    for fold, losses in enumerate(fold_results['losses']):
        ax1.plot(losses, label=f'Fold {fold+1}', linewidth=2)
    ax1.set_title('Training Loss Progress', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Accuracy 진행률
    for fold, accs in enumerate(fold_results['accuracies']):
        ax2.plot(accs, label=f'Fold {fold+1}', linewidth=2)
    ax2.set_title('Validation Accuracy Progress', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. F1 Score 진행률
    for fold, f1s in enumerate(fold_results['f1_scores']):
        ax3.plot(f1s, label=f'Fold {fold+1}', linewidth=2)
    ax3.set_title('F1 Score Progress', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('F1 Score')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # 4. 최종 성능 비교
    final_metrics = ['Accuracy', 'F1-Score', 'Precision', 'Recall']
    final_values = [
        np.mean(fold_results['final_accuracies']),
        np.mean(fold_results['final_f1s']),
        np.mean(fold_results['final_precisions']),
        np.mean(fold_results['final_recalls'])
    ]

    bars = ax4.bar(final_metrics, final_values,
                   color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
    ax4.set_title('Final Performance Metrics', fontsize=14, fontweight='bold')
    ax4.set_ylabel('Score')
    ax4.set_ylim(0, 1)

    # 막대 위에 수치 표시
    for bar, value in zip(bars, final_values):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_experiment_comparison(experiment_results, save_path=None):
    """실험별 성능 비교 그래프"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # 실험명과 결과 추출
    experiments = [exp['name'] for exp in experiment_results]
    accuracies = [exp['avg_accuracy'] for exp in experiment_results]
    f1_scores = [exp['avg_f1'] for exp in experiment_results]

    x = np.arange(len(experiments))
    width = 0.35

    # 1. Accuracy vs F1 비교
    bars1 = ax1.bar(x - width/2, accuracies, width, label='Accuracy',
                    color='#FF6B6B', alpha=0.8)
    bars2 = ax1.bar(x + width/2, f1_scores, width, label='F1-Score',
                    color='#4ECDC4', alpha=0.8)

    ax1.set_xlabel('Experiments', fontweight='bold')
    ax1.set_ylabel('Score', fontweight='bold')
    ax1.set_title('Performance Comparison Across Experiments',
                  fontsize=14, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(experiments, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)

    # 수치 표시
    for bar, value in zip(bars1, accuracies):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    for bar, value in zip(bars2, f1_scores):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontsize=9)

    # 2. 레이더 차트 (마지막 두 실험 비교)
    if len(experiment_results) >= 2:
        categories = ['Accuracy', 'F1-Score', 'Precision', 'Recall']

        # 마지막 두 실험 데이터
        exp1 = experiment_results[-2]
        exp2 = experiment_results[-1]

        values1 = [exp1['avg_accuracy'], exp1['avg_f1'],
                  exp1.get('avg_precision', 0.7), exp1.get('avg_recall', 0.7)]
        values2 = [exp2['avg_accuracy'], exp2['avg_f1'],
                  exp2.get('avg_precision', 0.7), exp2.get('avg_recall', 0.7)]

        # 각도 계산
        angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False)
        values1 += values1[:1]  # 원형으로 닫기
        values2 += values2[:1]
        angles = np.concatenate((angles, [angles[0]]))

        ax2 = plt.subplot(122, projection='polar')
        ax2.plot(angles, values1, 'o-', linewidth=2, label=exp1['name'], color='#FF6B6B')
        ax2.fill(angles, values1, alpha=0.25, color='#FF6B6B')
        ax2.plot(angles, values2, 'o-', linewidth=2, label=exp2['name'], color='#4ECDC4')
        ax2.fill(angles, values2, alpha=0.25, color='#4ECDC4')

        ax2.set_xticks(angles[:-1])
        ax2.set_xticklabels(categories)
        ax2.set_ylim(0, 1)
        ax2.set_title('Performance Radar Chart', fontweight='bold', pad=20)
        ax2.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_attention_heatmap(attention_weights, save_path=None):
    """Attention weight 히트맵"""
    fig, ax = plt.subplots(figsize=(10, 8))

    # 히트맵 생성
    sns.heatmap(attention_weights, annot=True, cmap='viridis',
                cbar_kws={'label': 'Attention Weight'}, ax=ax)

    ax.set_title('FlexAttention Weight Visualization',
                 fontsize=16, fontweight='bold')
    ax.set_xlabel('Patch Position', fontweight='bold')
    ax.set_ylabel('Patient Sample', fontweight='bold')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

# 사용 예시
"""
# 훈련 후 결과 시각화
plot_training_progress(training_results, 'training_progress.png')

# 실험들 비교
plot_experiment_comparison([exp1_results, exp2_results, exp4_results],
                          'experiment_comparison.png')

# Attention 시각화
plot_attention_heatmap(attention_data, 'attention_heatmap.png')
"""

print("🎨 예쁜 그래프 코드 준비 완료!")

# part 11 논문용 실험 코드드

In [ ]:
# # ========================================================================
# # 논문 실험용 다양한 설정들
# # ========================================================================

# # 1. Class Weight 추가 (Part 8에 추가)
# def get_class_weights(patient_labels):
#     """클래스 불균형 해결용 가중치 계산"""
#     from collections import Counter
#     class_counts = Counter(patient_labels)
#     total = len(patient_labels)

#     # 역비례 가중치
#     weight_0 = total / (2 * class_counts[0])  # 다수 클래스는 낮은 가중치
#     weight_1 = total / (2 * class_counts[1])  # 소수 클래스는 높은 가중치

#     weights = torch.tensor([weight_0, weight_1]).to(device)
#     print(f"🏷️ Class weights: {weights}")
#     return weights

# # Part 8에서 Loss function 부분을 이렇게 바꾸기:
# # class_weights = get_class_weights(patient_labels)
# # criterion = nn.CrossEntropyLoss(weight=class_weights)

# # ========================================================================

# # 2. Focal Loss 구현 (Part 8에 추가)
# class FocalLoss(nn.Module):
#     """Focal Loss for addressing class imbalance"""
#     def __init__(self, alpha=1, gamma=2, reduce=True):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.reduce = reduce

#     def forward(self, inputs, targets):
#         ce_loss = F.cross_entropy(inputs, targets, reduce=False)
#         pt = torch.exp(-ce_loss)
#         focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss

#         if self.reduce:
#             return torch.mean(focal_loss)
#         else:
#             return focal_loss

# # Part 8에서 사용: criterion = FocalLoss(alpha=1, gamma=2)

# # ========================================================================

# # 3. 실험별 설정들 (Part 9에서 바꿔가며 사용)

# # 실험 1: 현재 진행 중 (큰 모델)
# EXPERIMENT_1_CONFIG = {
#     'name': 'Large Model',
#     'num_folds': 5,
#     'num_epochs': 20,
#     'batch_size': 1,
#     'accumulation_steps': 4,
#     'learning_rate': 1e-4,
#     'feature_dim': 512,
#     'num_heads': 8,
#     'patches_per_megapatch': 16,
#     'max_megapatches': 10,
#     'loss_type': 'cross_entropy',
#     'extractor_type': 'resnet18'
# }

# # 실험 2: Class Weight 추가
# EXPERIMENT_2_CONFIG = {
#     'name': 'Large Model + Class Weight',
#     'num_folds': 5,
#     'num_epochs': 15,
#     'batch_size': 1,
#     'accumulation_steps': 4,
#     'learning_rate': 1e-4,
#     'feature_dim': 512,
#     'num_heads': 8,
#     'patches_per_megapatch': 16,
#     'max_megapatches': 10,
#     'loss_type': 'weighted_cross_entropy',
#     'extractor_type': 'resnet18'
# }

# # 실험 3: Focal Loss
# EXPERIMENT_3_CONFIG = {
#     'name': 'Large Model + Focal Loss',
#     'num_folds': 5,
#     'num_epochs': 15,
#     'batch_size': 1,
#     'accumulation_steps': 4,
#     'learning_rate': 5e-5,  # 더 작은 learning rate
#     'feature_dim': 512,
#     'num_heads': 8,
#     'patches_per_megapatch': 16,
#     'max_megapatches': 10,
#     'loss_type': 'focal_loss',
#     'extractor_type': 'resnet18'
# }

# # 실험 4: 최적화된 설정
# EXPERIMENT_4_CONFIG = {
#     'name': 'Optimized FlexAttention',
#     'num_folds': 5,
#     'num_epochs': 25,
#     'batch_size': 1,
#     'accumulation_steps': 4,
#     'learning_rate': 3e-5,
#     'feature_dim': 512,
#     'num_heads': 8,
#     'patches_per_megapatch': 20,  # 더 많은 패치
#     'max_megapatches': 12,        # 더 많은 메가패치
#     'loss_type': 'focal_loss',
#     'extractor_type': 'resnet18'
# }

# # ========================================================================

# # 4. 실험 실행 함수 (Part 8 수정)
# def train_flexattention_experiment(patient_data, experiment_config):
#     """실험별 설정으로 훈련 실행"""

#     print(f"🧪 실험 시작: {experiment_config['name']}")

#     # Loss function 선택
#     if experiment_config['loss_type'] == 'weighted_cross_entropy':
#         patient_labels = [patient_data[pid].get('t_label', 0) for pid in patient_data.keys()]
#         class_weights = get_class_weights(patient_labels)
#         criterion = nn.CrossEntropyLoss(weight=class_weights)
#     elif experiment_config['loss_type'] == 'focal_loss':
#         criterion = FocalLoss(alpha=1, gamma=2)
#     else:
#         criterion = nn.CrossEntropyLoss()

#     # 모델 설정
#     model = FlexAttentionPatientMIL(
#         feature_dim=experiment_config['feature_dim'],
#         num_classes=2,
#         num_heads=experiment_config['num_heads'],
#         num_sa_layers=1,
#         num_fa_layers=1,
#         dropout=0.1,
#         extractor_type=experiment_config['extractor_type']
#     )

#     # Dataset 설정에서 patches와 megapatches 적용
#     # (이 부분은 실제 훈련 함수에서 experiment_config 값 사용)

#     print(f"✅ {experiment_config['name']} 설정 완료")

#     # 기존 train_flexattention_model_with_checkpoints 함수 호출
#     # (실제 구현은 기존 함수 수정 필요)

# # ========================================================================

# # 5. 결과 비교 및 저장
# def save_experiment_results(results_list, save_path):
#     """실험 결과들을 비교표로 저장"""
#     import pandas as pd

#     comparison_data = []
#     for result in results_list:
#         comparison_data.append({
#             'Experiment': result['name'],
#             'Accuracy': f"{result['avg_accuracy']:.3f}",
#             'F1-Score': f"{result['avg_f1']:.3f}",
#             'AUC': f"{result.get('avg_auc', 0):.3f}",
#             'Feature_Dim': result['feature_dim'],
#             'Num_Heads': result['num_heads'],
#             'Loss_Type': result['loss_type']
#         })

#     df = pd.DataFrame(comparison_data)
#     df.to_csv(save_path, index=False)
#     print(f"📊 실험 결과 저장: {save_path}")

#     return df

# # ========================================================================

# # 6. 사용법
# """
# 실험 순서:

# 1. 현재 실험 완료 후 결과 저장
# 2. EXPERIMENT_2_CONFIG로 TRAINING_CONFIG 교체 후 실행
# 3. EXPERIMENT_3_CONFIG로 교체 후 실행
# 4. EXPERIMENT_4_CONFIG로 교체 후 실행
# 5. 모든 결과 비교 및 논문 테이블 생성

# 각 실험마다 Part 9에서 TRAINING_CONFIG를 해당 실험 config로 바꾸고
# Part 10 실행하면 됩니다.
# """

# print("🧪 실험 설정 코드 준비 완료!")
# print("실험 1 완료 후 Part 9에서 TRAINING_CONFIG를 EXPERIMENT_2_CONFIG로 바꿔주세요!")

🚀 초고속 테스트 설정 준비!
   예상 시간: 30분-1시간
   2-fold × 2-epoch = 총 4번만 훈련
